# Reproducibility of drought indicators in the ERA5–Drought dataset

Production date: 2026-02-xx

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Dataset version: 1.0.

Produced by: Enis Gerxhalija, Olivier Burggraaff (National Physical Laboratory).

## 🌍 Use case: Retrieving drought indicators from the ERA5-Drought dataset 

## ❓ Quality assessment question
* **Are the drought indicators in the ERA5-Drought dataset consistent with and reproducible from ERA5 data?**
* **Are the drought indicators in the ERA5-Drought dataset presented in a format that ensures optimal usability for users?**

Drought and extreme precipitation have far-reaching environmental, societal, and economic impacts.
In the United Kingdom, the record-breaking hot and dry spring and summer of 2025 caused harvest losses worth more than £800 million [[ECIU+25](https://eciu.net/analysis/reports/2025/estimated-financial-losses-faced-by-uk-farmers-due-dry-weather-impacts-on-key-arable-crops)].
An 18-month drought in Brazil in 2023–24,
the most severe since monitoring began in 1954,
led to 720 health centres in affected areas becoming non-operational [[UNICEF+24](https://www.unicef.org/media/165191/file/LACR-Flash-Update-11-November-2024.pdf)].
Finally, extreme rainfall killed hundreds of people and caused billions of € of damages in Spain in 2024 [[Franch-Pardo+25](https://doi.org/10.48088/ejg.i.fra.16.2.286.297)].
Climate change is thought to be the primary driver behind the increase in drought and extreme precipitation events since the 1950s [[IPCC+23](https://doi.org/10.59327/IPCC/AR6-9789291691647)],
and this trend is expected to continue into the future.

Given these impacts,
monitoring drought and extreme precipitation is vital.
Several quantitative proxies have been developed to aid in this monitoring,
generally based on
(a combination of)
total precipitation, soil moisture, evapotranspiration, and surface (air) temperature.
Two widely-employed indices are the 
_Standardised Precipitation Index (SPI)_ [[McKee+93](https://climate.colostate.edu/pdfs/relationshipofdroughtfrequency.pdf)]
and
_Standardised Precipitation-Evapotranspiration Index (SPEI)_ [[Vicente-Serrano+10](https://doi.org/10.1175/2009JCLI2909.1)].
Both operate on the same principle,
namely quantifying the amount of precipitation
over a given time frame at a given location
relative to its historical climatology.
For example,
an SPI value of +1 corresponds to a precipitation that is 1 standard deviation above the mean,
for that site and time frame.
This probabilistic approach lends itself to statements on the occurrence rate of extreme events [[McKee+93](https://climate.colostate.edu/pdfs/relationshipofdroughtfrequency.pdf)].
SPEI expands on SPI by including not just gains from precipitation, but also losses due to evapo(transpi)ration.
SPI and SPEI can be evaluated over different _accumulation periods_ to probe phenomena at different time scales:
* 1, 3 months: soil moisture, flow in small creeks.
* 6, 12 months: reservoir storage, stream flow.
* 24, 36, 48 months: groundwater recharge, reduced reservoir.
**Reference for these?**

Of course,
proxies like SPI and SPEI depend on reliable input data in the form of historical meteorological time series.
Weather stations can provide these data with high accuracy and precision for specific sites,
but their coverage is sparse and their data are not always interoperable.
Reanalyses,
which integrate observations and forecast modelling,
can provide similar data consistently with long-term global coverage.
For example, ECMWF's fifth-generation reanalysis _ERA5_ provides meteorological data on a global ~31 km grid
going back to 1940 [[Soci+24](https://doi.org/10.1002/qj.4803), [Hersbach+20](https://doi.org/10.1002/qj.3803)].

The Copernicus Climate Change Service (C3S)
now provides pre-calculated SPI and SPEI indices derived from ERA5
in the ERA5–Drought dataset [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)],
[available from the Climate Data Store](https://doi.org/10.24381/9bea5e16).
This derived dataset can be a valuable resource for applications in many sectors,
since it can be used out of the box,
freeing users from the need to find and process the underpinning meteorological data themselves.
ERA5–Drought provides SPI and SPEI for 7 different accumulation periods,
interpolated to a 0.25° × 0.25° grid.
It includes ERA5's deterministic reanalysis as well as 10 propagated members of the ERA5-EDA ensemble.
The latter can be used to quantify the uncertainty in SPI or SPEI resulting from uncertainty in the input data.
Moreover, ERA5–Drought provides multiple quality flags that can be used to filter data that do not meet the requirements for SPI or SPEI to be applicable.

This quality assessment tests the consistency of ERA5–Drought with the underpinning ERA5 data in terms of its reproducibility.
By manually reproducing SPI, SPEI, and the associated quality indicators from ERA5 data and comparing the results,
we assess whether ERA5–Drought can indeed be used as a more convenient alternative to ERA5 for monitoring drought and extreme precipitation.
Furthermore,
any discrepancies found in the comparison shed light on caveats in the use of ERA5–Drought specifically and drought indicators broadly,
such as the underlying assumptions.
This notebook provides code for reproducing SPI and SPEI from ERA5 data and can be a jumping-off point for further analysis by the reader.

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Conclusion 1
* Conclusion 2

## 📋 Methodology

This quality assessment tests the consistency between drought indices retrieved from the [C3S ERA5-Drought dataset] and their equivalents calculated from the origin datasets, as well as the reproducibility and usability of said dataset.

We will examine the SPI and SPEI drought indicators calculated from the following datasets:

(include table here of the parameter, description of that parameters, and the origin dataset)

The analysis and results are organised in the following steps, which are detailed in the sections below:

**[](section-code_setup)**
 * Import all required libraries.
 * Define helper functions.

**[](section-general_setup)**
 * Define scope of analysis (time, location).
 * Set up CDS downloads for ERA5 and ERA5-Drought.
 * Download land-sea mask.

**[](section-spi)**
 * Define SPI Indicator
 * Download ERA5 precipitation
 * Accumulate
 * Calculate SPI
 * Calculate quality flags from ERA5 data
 * Download quality flags from ERA5-Drought
 * Compare quality flags
 * Download ERA5-Drought SPI
 * Comparison

**[](section-spei)**
 * Define SPEI Indicator
 * Download ERA5 potential evaporation
 * Accumulate
 * Calculate SPEI
 * Calculate quality flags from ERA5 data
 * Download quality flags from ERA5-Drought
 * Compare quality flags
 * Download ERA5-Drought SPEI
 * Comparison

**[](section-ensemble)**

**[](section-conclusion)**


## 📈 Analysis and results

(section-code_setup)=
### 1. Code setup
```{note}
This notebook uses [earthkit](https://github.com/ecmwf/earthkit) for downloading ([earthkit-data](https://github.com/ecmwf/earthkit-data)) and visualising ([earthkit-plots](https://github.com/ecmwf/earthkit-plots)) data. Because earthkit is in active development, some functionality may change after this notebook is published. If any part of the code stops functioning, please raise an issue on our GitHub repository so it can be fixed.
```

#### Import required libraries
In this section, we import all the relevant packages needed for running the notebook.

In [ ]:
# Input / Output
from pathlib import Path
import earthkit.data as ekd
import warnings

# General data handling
import numpy as np
import pandas as pd
import xarray as xr
from functools import partial
from itertools import batched

# Analysis
from dask.array import nanmedian, ravel
from scipy import stats
from sklearn.metrics import confusion_matrix

# Visualisation
import earthkit.plots as ekp
from earthkit.plots.styles import Style
from cartopy import crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm, ListedColormap
from matplotlib.cm import ScalarMappable
plt.rcParams["grid.linestyle"] = "--"
from tqdm import tqdm  # Progress bars

# Visualisation in Jupyter book -- automatically ignored otherwise
try:
    from myst_nb import glue
except ImportError:
    glue = None

# Type hints
from typing import Any, Callable, Iterable, Optional
from scipy.stats import rv_continuous as Distribution
from pandas.io.formats.style import Styler as pdStyler
from earthkit.plots.geo.domains import Domain
AnyDomain = (Domain | str)

#### Helper functions
This section defines some functions and variables used in the following analysis, allowing code cells in later sections to be shorter and ensuring consistency.

##### Data downloading & (pre-)processing

The following functions handle downloading data in specific circumstances, e.g. a geographical or temporal subset:

In [ ]:
# Geographical
def request_data_for_one_site(*, lat, lon, half_width=0.25):
    """ Return a CDS request for a bounding box around a point (lon, lat). """
    north, south = lat + half_width, lat - half_width
    east , west  = lon + half_width, lon - half_width
    box = [north, west, south, east]
    box = [round(x, 2) for x in box]  # Round all coordinates to 2 digits
    request_box = {"area": box}
    return request_box

# Handling CDS size limits
def batch_requests(main_request: dict, *, batch_key: str="year", n: int=20) -> list[dict]:
    """ Take a big request (e.g. ERA5–Drought for all years) and separate it into smaller ones (size `n`). """
    full_range = main_request[batch_key]  # e.g. [1940, 1941, ..., 2024]
    batched_range = batched(full_range, n)  # e.g. [1940, ..., 1959], [1960, ..., 1979], ...
    subrequests = [main_request | {batch_key: batch} for batch in batched_range]  # create corresponding CDS requests
    return subrequests

The following functions ensure consistency in dimensions, which have inconsistent names (e.g. `valid_time` vs. `time`) and definitions (e.g. longitude 0 to 360 or –180 to 180) between ERA5 and ERA5-Drought:

In [ ]:
def rename_era5_dimensions(data: xr.Dataset) -> xr.Dataset:
    """
    Rename dimensions
    from ERA5 (valid_time, latitude, longitude)
    to ERA5–Drought (time, lat, lon)
    format.
    """
    data = data.rename({"valid_time": "time",
                        "latitude": "lat",
                        "longitude": "lon",
                       })
    return data

def longitude_360_to_180(data: xr.Dataset, longitude_dimension="lon") -> xr.Dataset:
    """ Convert longitude from 0..360 to -180..180 format. """
    longitude_new = ((data[longitude_dimension] - 180) % 360) - 180
    data = data.assign_coords({longitude_dimension: longitude_new})
    data = data.sortby(longitude_dimension)#.squeeze()
    return data

The following functions handle unit conversions, e.g. m to mm precipitation:

In [ ]:
# Unit conversion
def convert_unit(dataset: xr.Dataset, key: str, conversion: Callable, new_unit: str) -> None:
    """ Convert the units of dataset[key] to new_unit using a conversion function (e.g. lambda x: x*1000 for m to mm), in-place. """
    # Metadata handling
    metadata_old = dataset[key].attrs
    metadata_new = metadata_old | {"units": new_unit}

    # Apply changes
    dataset[key] = conversion(dataset[key]).assign_attrs(**metadata_new)

    return dataset  # Redundant because modified in-place

convert_m2mm = partial(convert_unit, conversion=(lambda x: x*1000), new_unit="mm")  # meter -> millimeter

The following function handles the overall pre-processing of ERA5 data: renaming dimensions, converting units, and adjusting longitudes:

In [ ]:
def preprocess_era5(data: xr.Dataset) -> xr.Dataset:
    """ Pre-process ERA5 data (precipitation and/or evapotranspiration) into the right format for analysis. """
    # Dimensions
    data = rename_era5_dimensions(data)
    data = longitude_360_to_180(data, "lon")  # Convert longitude from 0..360 to –180..180

    # Units
    for var in data.data_vars:  # Assume all data_vars need to be converted to mm
        data = convert_m2mm(data, var)  # Convert from m to mm

    # Re-chunk for speed gain in fitting
    data = data.chunk({"time": -1, "lat": 103, "lon": 360,})

    return data

The following function calculates the water balance `wb` from the total precipitation and potential evaporation ([](section-spei)):

In [ ]:
def calculate_waterbalance(data: xr.Dataset, *,
                           precipitation_variable: str="tp", evaporation_variable: str="pev",
                           waterbalance_variable: str="wb") -> xr.Dataset:
    """
    Calculate the water balance from precipitation and evaporation.
    Assumes the ECMWF sign convention where downward flux is positive and upward flux is negative,
    i.e. WB = TP + PEV.
    """
    # Calculate water balance
    waterbalance = data[precipitation_variable] + data[evaporation_variable]

    # Apply variable name, metadata, and convert to dataset
    waterbalance = waterbalance.rename(waterbalance_variable)
    waterbalance = waterbalance.assign_attrs({"units": data[precipitation_variable].units,
                                              "long_name": "Water balance",
                                             })
    waterbalance = waterbalance.to_dataset()
    
    return waterbalance

The following functions aid in processing quality flag data from ERA5–Drought into a standard format:

In [ ]:
def preprocess_era5drought_qualityflag(data: dict[Any, ekd.FieldList], variable_out: Optional[str]=None, *,
                                       month_dimension: str="month", time_dimension: str="time") -> xr.Dataset:
    """ 
    Turn a dict of earthkit FieldLists with ERA5–Drought quality flags, organised by accumulation period, into one big xr.Dataset.
    Rename the variables according to the dict keys; optionally using `variable_out` (e.g. SPEI -> SPEI1, SPEI3, etc).
    Convert a 12-month time dimension (2020-01-01, 2020-02-01, ...) into a month dimension (1, 2, ...).
    """
    # Convert field lists to xarray Datasets
    data = {key: ds.to_xarray(compat="equals")
            for key, ds in data.items()}

    # Rename variables to include keys (e.g. accumulation periods)
    first_fieldlist = next(iter(data.values()))
    data_var = next(iter(first_fieldlist.data_vars))
    if not variable_out:  # If not manually specified, append keys to existing variable name
        variable_out = data_var
    
    data = [ds.rename_vars({data_var: f"{variable_out}{key}"})
            for key, ds in data.items()]

    # Merge into one
    data = xr.merge(data, compat="equals")

    # Change time dimension to months
    data = data.assign_coords({month_dimension: data[time_dimension].dt.month})
    data = data.swap_dims({time_dimension: month_dimension})
    
    return data

The following functions restructure the ensemble members in the ERA5-Drought dataset along a new dimension, as in ERA5:

In [ ]:
def add_number_dimension(data: xr.Dataset, *, n_members: int=10,
                         time_dimension: str="time", ensemble_dimension: str="number") -> xr.Dataset:
    """
    Convert a dataset
    from ERA5-Drought format (10-duplicate `time` dimension)
    to ERA5 format (`number` dimension for ensemble members)
    """
    # Find unique times and use these to generate datasets for successive members
    member_numbers = np.arange(n_members)
    _, index = np.unique(data[time_dimension], return_index=True)
    data = [data.isel({time_dimension: index + i}) for i in member_numbers]

    # Combine into one dataset
    data = xr.concat(data, dim=ensemble_dimension).assign_coords(number=member_numbers)

    # Rechunk for memory efficiency
    data = data.chunk({ensemble_dimension: n_members, time_dimension: 48, "lat": 360, "lon": 103,})

    return data

##### Accumulation periods
The following cells contain constants and functions used in accumulating variables (e.g. precipitation) over time:

In [ ]:
# Define constants such as the accumulation periods to use
ACCUMULATION_PERIODS = [1, 3, 6, 12, 24, 36, 48]  # Months
MONTHS = range(1, 13)  # January to December (inclusive)
MONTHS_NAMED = {
     1:  "January", 2:  "February", 3:  "March",
     4:  "April",   5:  "May",      6:  "June",
     7:  "July",    8:  "August",   9:  "September",
    10: "October", 11: "November", 12:  "December",
}

# Perform accumulation
def accumulate(data: xr.Dataset, var: str, *,
               accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS, time_dimension: str="time") -> xr.Dataset:
    """
    Accumulate the given variable `var` (e.g. total precipitation) over different accumulation periods.
    """
    # Find the number of days in each month
    time_index = pd.to_datetime(data[time_dimension])  # Convert time dimension to datetime format
    days_in_month = xr.DataArray(time_index.days_in_month, coords={time_dimension: data[time_dimension]})  # Aligned with `data` coordinates

    # Monthly mean `var` → Monthly total `var`
    monthly_total = data[var] * days_in_month

    ## Accumulate over periods (rolling windows)
    result = data.copy()
    for period in accumulation_periods:
        rolling_sum = monthly_total.rolling({time_dimension: period}, center=False).sum()  # Rolling sum
        result[f"{var}{period}"] = rolling_sum  # Assign variable in same format as ERA5–Drought (e.g. SPI6)

    # Reorganise
    result = result.drop_vars([var])  # Drop original var
    result = result.chunk(data.chunksizes)  # Rechunk like input data

    return result

The following functions calculate the probability
(regular and accounting for the centre of mass)
of months with zero (accumulated) precipitation:

In [ ]:
def number_of_months_with_zero_precipitation(data: xr.Dataset, *,
                                             round_to: int=2, threshold: float=0.,
                                             time_dimension: str="time") -> xr.Dataset:
    """
    Find the number of months with zero precipitation (and total months) in a given dataset.
    Rounds to `round_to` decimals (default 2, e.g. 0.01 mm).
    Zero precipitation means <= `threshold` (0. by default).
    Applied per calendar month and to all variables (e.g. accumulation periods).
    """
    # Round data, to 2 decimals by default
    data = data.round(round_to)

    # Find months below threshold
    is_zero = (data <= threshold)

    # Group by calendar month and find total number
    data_by_calendar_month = is_zero.groupby(is_zero[time_dimension].dt.month)
    n_zero  = data_by_calendar_month.sum(dim=time_dimension).astype(np.uint8)
    n_total = data_by_calendar_month.count(dim=time_dimension).astype(np.uint8)

    return n_total, n_zero

def probability_of_zero_precipitation(data: xr.Dataset, **kwargs) -> xr.Dataset:
    """ Calculate the simple probability of zero precipitation in the given dataset, per calendar month. """
    # Find number of months (total / zero precipitation)
    n_total, n_zero = number_of_months_with_zero_precipitation(data, **kwargs)

    # Calculate simple probability
    p_zero = n_zero / (n_total + 1)

    return p_zero
   
def probability_of_zero_precipitation_weighted(data: xr.Dataset, **kwargs) -> xr.Dataset:
    """ Calculate the weighted probability of zero precipitation in the given dataset, per calendar month. """
    # Find number of months (total / zero precipitation)
    n_total, n_zero = number_of_months_with_zero_precipitation(data, **kwargs)    

    # Calculate weighted probability
    p_zero = xr.where(
        n_zero > 0,                          # Condition: More than 0 months with zero precipitation
        (n_zero + 1) / (2 * (n_total + 1)),  # If condition met: weighted probability
        0                                    # If condition not met: just 0
    )

    return p_zero

def zero_precip_rejection(prob_ds: xr.Dataset, ls_mask):
    """
    Compute zero-precipitation rejection masks from monthly probabilities and
    apply a land–sea mask.

    Logic:
      - Mark a month as 'rejected' if probability >= 0.33 (1 = reject, 0 = keep).
      - Aggregate across the 'month' dimension to get the count per grid cell.
      - Apply a land–sea mask so ocean pixels are set to NaN.
      - Return both the monthly rejection field and the aggregated counts across time.
    """
    
    rejection = xr.where(
            prob_ds >= 0.33,
            1,
            0
        )
    
    # Aggregate across time 
    rejection_agg = rejection.sum(dim=[d for d in ["month", "time"] if d in rejection.dims])

    # Apply land-sea mask to monthly
    rejection_ds = rejection.where(ls_mask["lsm"])
    
    # Apply land-sea mask to aggregated
    rejection_agg_ds = rejection_agg.where(ls_mask["lsm"])

    return rejection_ds, rejection_agg_ds

##### Calculating SPI and SPEI
The following functions fit the appropriate distributions
(gamma, generalised logistic)
and calculate corresponding CDF and SPI / SPEI values:

In [ ]:
# General fitting function
def fit_distributions(reference_data: xr.Dataset, distribution: Distribution, *,
                      time_dimension: str="time") -> xr.Dataset:
    """
    Fit distributions (e.g. gamma) for each month and accumulation period using xarray parallelisation.
    Data are assumed to have been sliced to the reference period.
    """
    # Define fitting function for the given distribution
    def fit(y):
        y = y[np.isfinite(y)]
        try:
            params = distribution.fit(y)
        except:  # If fitting fails, return NaN
            # Should only catch a specific Exception type
            params = [np.nan]*(distribution.numargs+2)
        finally:  # Always convert result (values or NaN) to desired format
            params = np.stack(params, axis=-1)  # Extend with axis for stats (alpha, loc, scale, ...)
        return params

    # Split dataset by month
    reference_data_by_month = reference_data.groupby(reference_data[time_dimension].dt.month)

    # Apply fitting function by month
    params = xr.apply_ufunc(fit, reference_data_by_month,
                            input_core_dims=[[time_dimension]], output_core_dims=[["stat"]],
                            vectorize=True, dask="parallelized",
                            dask_gufunc_kwargs={"output_sizes": {"stat": distribution.numargs+2}},  # e.g. 3 for gamma (alpha, loc, scale)
                            output_dtypes=[np.float64],
                           )
    params = params.chunk({"month": -1})

    return params

# Fitting functions for SPI, SPEI distributions specifically
fit_monthly_spi  = partial(fit_distributions, distribution=stats.gamma)
fit_monthly_spei = partial(fit_distributions, distribution=stats.genlogistic)

The following functions apply the fitted distributions to calculate CDF values for observed data:

In [ ]:
# Helper function: Clip the CDF to avoid infinities
def clip_cdf(cdf: xr.Dataset, threshold: float=1e-16) -> xr.Dataset:
    """ Clip CDF values to `threshold` on both sides. """
    cdf_clipped = cdf.clip(threshold, 1 - threshold)
    return cdf_clipped

# Main function: Compute CDF values for observed data
def compute_cdf(data: xr.Dataset, parameters: xr.Dataset, distribution: Distribution, *,
                month_dimension: str="month", time_dimension: str="time") -> xr.Dataset:
    """
    For observed `data`, compute the corresponding cumulative distribution function (CDF)
    values for the given `distribution` and `parameters`.

    Assumes that `data` and `parameters` have compatible:
        Coordinates (except time)
        Variables (e.g. corresponding to accumulation periods)
    """
    # Create a month dimension for broadcasting with the one in parameters
    data_month = data[time_dimension].dt.month.rename(month_dimension)

    # Extract parameters
    nr_parameters = parameters.sizes["stat"]  # 3 for gamma / genlogistic
    parameters_extracted = [parameters.sel(stat=j).sel({month_dimension: data_month}) for j in range(nr_parameters)]

    # Calculate CDF values by month
    cdf = xr.apply_ufunc(distribution.cdf, data, *parameters_extracted,
                         input_core_dims=[[], [], [], []], output_core_dims=[[]],
                         vectorize=True, dask="parallelized",
                         output_dtypes=[np.float64],
                         keep_attrs=True
                        )
    cdf = clip_cdf(cdf)  # Clip extreme values to avoid infinities
    cdf = cdf.chunk(data.chunksizes) # Rechunk like input data

    return cdf

# Compute SPI, SPEI with preset distributions
compute_cdf_spi  = partial(compute_cdf, distribution=stats.gamma)
compute_cdf_spei = partial(compute_cdf, distribution=stats.genlogistic)

def adjust_cdf_for_zero_precipitation(cdf: xr.Dataset, data: xr.Dataset, *,
                                      time_dimension: str="time") -> xr.Dataset:
    """
    Adjust CDF for months with zero precipitation, following Stagge+15 (doi:10.1002/joc.4267) method.
    `cdf` can be for any time range, `data` should be for the reference period only.
    """ 
    # Calculate probability of zero precipitation
    p_zero = probability_of_zero_precipitation_weighted(data)
    p_zero = p_zero.chunk({"month": -1})  # Rechunk for efficiency

    # Adjust CDF by calendar month
    def adjust_cdf_by_calendar_month(cdf_one_month: xr.Dataset) -> xr.Dataset:
        this_month = cdf_one_month[time_dimension][0].month.values # Cannot use GroupBy label inside .map() unfortunately
        p_zero_this_month = p_zero.sel({"month": this_month})
        cdf_adjusted = p_zero_this_month + (1 - p_zero_this_month) * cdf_one_month
        return cdf_adjusted

    cdf_by_calendar_month = cdf.groupby(cdf[time_dimension].dt.month)
    cdf_adjusted = cdf_by_calendar_month.map(adjust_cdf_by_calendar_month)  
    # Alternative approach: Broadcast p_zero against cdf instead of using GroupBy (not implemented here)

    # Reorganise for efficiency
    cdf_adjusted = cdf_adjusted.chunk(cdf.chunksizes)

    return cdf_adjusted

The following functions calculate the SPI/SPEI index based on CDF values:

In [ ]:
# Main function: Convert CDF values to indexes
def cdf_to_index(cdf: xr.Dataset, *, var: Optional[str]=None, index_name: Optional[str]=None) -> xr.Dataset:
    """ Transform CDF values to a standard normal distribution. """
    cdf = clip_cdf(cdf)  # Clip extreme values to avoid infinities

    # Convert CDF to index
    index_values = xr.apply_ufunc(stats.norm.ppf, cdf, 0.0, 1.0,  
                                  input_core_dims=[[], [], []], output_core_dims=[[]],
                                  vectorize=True, dask="parallelized",
                                  output_dtypes=[np.float64],
                                  keep_attrs=True,
                                 )

    # Adjust variable names to ERA5–Drought format (e.g. SPI6) if desired
    if var and index_name:
        rename_variables = {datavar: datavar.replace(var, index_name) for datavar in cdf.data_vars if var in datavar}
        index_values = index_values.rename_vars(rename_variables)

    return index_values

# Compute SPI, SPEI with preset names
cdf_to_spi  = partial(cdf_to_index, var="tp", index_name="SPI")
cdf_to_spei = partial(cdf_to_index, var="wb", index_name="SPEI")

For convenience, we also provide the following functions that wrap the entire process from data to SPI/SPEI into one:

In [ ]:
# Full SPI pipeline
def calculate_spi_from_era5(data: xr.Dataset, *, reference_window: dict[str, slice],
                            do_preprocessing=False, evaluation_window: Optional[dict[str, slice]]=None,
                            precipitation_variable: str="tp",
                            accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS,
                           ) -> xr.Dataset:
    """
    Calculate SPI from ERA5 precipitation data, including all steps.
    Input should be ERA5 data in xarray format, either
        freshly downloaded (`do_preprocessing=True`)
        or
        pre-processed (`do_preprocessing=False`).
    Some parameters can be adjusted,
        e.g. the name of the precipitation variable ("tp" by default).
        This should make the function easier to transfer to other datasets.
    """
    # Optional: pre-process to desired format
    if do_preprocessing:
        data = preprocess_era5(data)

    # Accumulate total precipitation
    data = accumulate(data, precipitation_variable, accumulation_periods=accumulation_periods)

    # Select data in reference window
    data_reference = data.sel(**reference_window)
    
    # Fit gamma distribution
    spi_parameters = fit_monthly_spi(data_reference)

    # Optional: Only calculate index in evaluation window
    if evaluation_window:
        data = data.sel(**evaluation_window)

    # Compute CDF time series
    cdf = compute_cdf_spi(data, spi_parameters)

    # Adjust CDF
    cdf_adjusted = adjust_cdf_for_zero_precipitation(cdf, data_reference)

    # Calculate SPI from adjusted CDF
    spi = cdf_to_spi(cdf_adjusted)
    spi = spi.chunk(data.chunksizes)

    return spi

# Full SPEI pipeline
def calculate_spei_from_era5(data: xr.Dataset, *, reference_window: dict[str, slice],
                             do_preprocessing=False, evaluation_window: Optional[dict[str, slice]]=None,
                             precipitation_variable: str="tp", evaporation_variable: str="pev",
                             accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS,
                            ) -> xr.Dataset:
    """
    Calculate SPEI from ERA5 precipitation and potential evaporation data, including all steps.
    Input should be ERA5 data in xarray format, either
        freshly downloaded (`do_preprocessing=True`)
        or
        pre-processed (`do_preprocessing=False`).
    Some parameters can be adjusted,
        e.g. the name of the precipitation variable ("tp" by default).
        This should make the function easier to transfer to other datasets.
    """
    # Optional: pre-process to desired format
    if do_preprocessing:
        data = preprocess_era5(data)

    # Calculate water balance
    data = calculate_waterbalance(data)

    # Accumulate water balance
    data = accumulate(data, "wb")

    # Select data in reference window
    data_reference = data.sel(**reference_window)

    # Fit generalised logistic distribution
    spei_parameters = fit_monthly_spei(data_reference)

    # Optional: Only calculate index in evaluation window
    if evaluation_window:
        data = data.sel(**evaluation_window)

    # Compute CDF time series
    cdf = compute_cdf_spei(data, spei_parameters)

    # Calculate SPI from CDF
    spei = cdf_to_spei(cdf)
    spei = spei.chunk(data.chunksizes)

    return spei

##### Categorising SPI and SPEI
The following cell defines categories for SPI and SPEI values, e.g. "severe drought":

In [ ]:
# Category format: [lower limit, upper limit, label, colour]
# Colours are picked from Fig. 4 in Keune+25 and labelled for convenience
# Limits are left-inclusive: [lower, upper)
# Extreme values of 100 / -100 should be treated as infinities but play nicer with some code than np.inf

CATEGORIES_SPI = [  # Approximates the GDO scheme
    (   2.00, 100,     "Extremely wet",            "#7a007b"),  # deepest purple 
    (   1.50,   2.00,  "Severely wet",             "#af51c3"),  # dark purple
    (   1.00,   1.50,  "Moderately wet",           "#eaccf8"),  # medium purple
    (   0.00,   1.00,  "Near‑normal / mildly wet", "#ffffff"),  # white
    (  -1.00,   0.00,  "Near‑normal / mildly dry", "#ffffff"),  # white
    (  -1.50,  -1.00,  "Moderately dry",           "#fffc03"),  # yellow
    (  -2.00,  -1.50,  "Severely dry",             "#feaa00"),  # orange
    (-100,     -2.00,  "Extremely dry",            "#ff0100"),  # red
]

CATEGORIES_SPEI = [
    (   2.33, 100,    "Extremely wet",  "#01148b"),  # very dark navy
    (   1.65,   2.33, "Severely wet",   "#1871de"),  # strong blue
    (   1.28,   1.65, "Moderately wet", "#14acf4"),  # medium blue
    (   0.84,   1.28, "Mildly wet",     "#00f2fe"),  # cyan
    (  -0.84,   0.84, "Near-normal",    "#9afa93"),  # light green
    (  -1.28,  -0.84, "Mildly dry",     "#fdc403"),  # yellow
    (  -1.65,  -1.28, "Moderately dry", "#f2631d"),  # orange
    (  -2.33,  -1.65, "Severely dry",   "#df2929"),  # red
    (-100,     -2.33, "Extremely dry",  "#8c1b1a"),  # dark red
]

The following functions categorise SPI / SPEI values into said categories:

In [ ]:
def _digitise_dataset(data: xr.Dataset, bin_edges: list[float], *, persist=True) -> xr.Dataset:
    """
    Digitise all variables in `data` according to a list of left bin edges, e.g. SPI categories ("extremely dry").
    In this notebook, do not call this function directly, but use the categorise_dataset wrapper.
    """
    # Perform digitisation
    data_digitised = xr.apply_ufunc(np.digitize, data,
                                    kwargs={"bins": bin_edges, "right": False}, 
                                    input_core_dims=[[]],
                                    vectorize=True, dask="parallelized",
                                    output_dtypes=[np.uint8],  # uint8 is small; not suitable for >256 categories
    )

    # Persist in memory if desired (default True because uint8s are small)
    if persist:
        data_digitised = data_digitised.persist()

    return data_digitised

def categorise_dataset(data: xr.Dataset, categories: Iterable[Iterable], **kwargs) -> xr.Dataset:
    """
    Categorise a dataset into bins defined by category thresholds.
    Extracts bin edges from `categories` and then digitises `data` accordingly.
    Handles NaNs by adding a dummy category (extremely high bin edge).
    """
    # Extract bin edges from categories
    # Last element is dropped to extend range to infinity (although values should be clipped anyway)
    bin_edges = [category[0] for category in categories[:-1]]

    # Add dummy category to catch NaNs
    bin_edges = [100000] + bin_edges

    # Apply digitisation
    data_categorised = _digitise_dataset(data, bin_edges, **kwargs)

    return data_categorised

##### Quality flags
The following functions create and apply quality flag tests, such as the probability of zero precipitation and the Shapiro–Wilk normality test:

In [ ]:
def xr_shapiro_test(spi_ds: xr.Dataset, reference_window: dict[str, slice],
                    accum_periods: Iterable[int]=ACCUMULATION_PERIODS,
                    months = range(1,13), index_name: Optional[str]=None ):
    
    """ Function to perform Shapiro-Wilks test on data within reference period
    Returns Shapiro-Wilks statistic along with probability of falling under normal distribution, along with significance value (1/0)."""
    
    spi_ref = spi_ds.sel(**reference_window)  

    time_dim = find_time_dim(spi_ds) # find time dimension.
        
    spi_ref = spi_ref.chunk({time_dim: -1}) 
    
    spi_ref = spi_ref.where(np.isfinite(spi_ref))  # Mask non-finite values before applying shapiro-wilks.

    spi_ref_by_month = spi_ref.groupby(f"{time_dim}.month")

    # Perform shapiro on xarray
    _ , pval = xr.apply_ufunc(stats.shapiro, spi_ref_by_month,
                                input_core_dims=[[time_dim]], output_core_dims=[[],[]],
                                vectorize=True, dask="parallelized",
                                output_dtypes=[np.float64, np.float64],
                                keep_attrs=True,
                               )
    
    normality = xr.where(pval < 0.05, 0, 1)  # Values < 0.05 → 0

    for p in accum_periods:
        normality = normality.rename({f"{index_name}{p}": f"significance_{p}"})

    month_ts = pd.to_datetime([f"2020-{int(m):02d}-01" for m in normality["month"].values])
    
    # Assign those timestamps to the existing 'month' dimension,
    # then rename the dimension from 'month' -> 'time'
    normality = (
        normality
        .assign_coords(month=("month", month_ts))
        .rename(month="time")
    )

    return pval, normality

def apply_sw_quality_mask(era5_quality: xr.Dataset, index_ds: xr.Dataset, indicator: str):
    """
    Apply significance-based quality masks to drought indicator datasets.
    """
    index_ds = safe_rename(index_ds)
    for period in ACCUMULATION_PERIODS:
        sig = era5_quality[f"significance_{period}"]
        sig = sig.assign_coords(time=sig.time.dt.month)
        mask = sig.sel(time=index_ds[f"{indicator}{period}"].time.dt.month)
        index_ds[f"{indicator}{period}"] = index_ds[f"{indicator}{period}"].where(mask.values == 1)
    return index_ds


def count_flagged_months(data: xr.Dataset, *, invert=True) -> xr.Dataset:
    """ For a dataset with quality flags for 12 months, return a new dataset with the number of months failing that flag. """
    sum_by_month = data.sum(dim="month").astype(np.uint8)  # Number of months *passing* the flag
    if invert:  # Make False for e.g. mismatched months
        sum_by_month = 12 - sum_by_month                   # Number of months *failing* the flag
    return sum_by_month

##### Statistics
The following functions calculate and display the difference (by month) between datasets:

In [ ]:
# Helper functions: Calculate statistics
def align_data_variables(data1: xr.Dataset, data2: xr.Dataset, var: Optional[str]="") -> tuple[xr.Dataset]:
    """
    Only look at variables that are in both datasets, following the order in `data2`.
    Optional: variables have to include `var`, e.g. "SPI".
    """
    matching_variables = [data_var for data_var in data2.data_vars
                          if data_var in data1.data_vars and var in data_var]
    data1, data2 = [data[matching_variables] for data in (data1, data2)]
    return data1, data2

def median_by_month(data: xr.Dataset, *, time_dimension: str="time") -> xr.Dataset:
    """ Calculate the median by month, flattening all other dimensions (time, lat, lon, etc.) """
    data_by_month = data.groupby(data[time_dimension].dt.month)
    median = data_by_month.quantile(0.5, dim=..., skipna=True)  # .quantile function has better dask support than .median
    median = median.drop_vars("quantile")  # Drop unneeded coordinate
    return median

# Main function: Calculate statistics
def comparison_monthly_statistics(data1: xr.Dataset, data2: xr.Dataset, *,
                                  time_dimension: str="time") -> pd.DataFrame:
    """
    Given two datasets, calculate a number of statistics for each variable and return the result in a table.
    """
    # Align data variables
    data1, data2 = align_data_variables(data1, data2)

    # Calculate difference
    difference = data2 - data1
    difference_absolute = xr.ufuncs.abs(difference)

    # Calculate monthly medians
    md  = median_by_month(difference,          time_dimension=time_dimension).to_pandas()
    mad = median_by_month(difference_absolute, time_dimension=time_dimension).to_pandas()

    # Combine into one DataFrame
    stats = pd.concat({"Δ": md, "|Δ|": mad}, axis=1)  # Combine
    stats = stats.swaplevel(0, 1, axis=1)             # Accumulation period on top, statistic below
    stats = stats[data2.data_vars]                    # Same order as inputs

    return stats

In [ ]:
# Helper function: Styling for statistics
def _add_matching_vertical_separators(styler: pdStyler, *,
                                      colour="#bbbbbb", width="1px") -> pdStyler:
    """
    Add vertical separators corresponding to the top level columns,
        e.g. SPI1 – SPI3 etc. but not Δ – |Δ| in comparison_monthly_statistics outputs.

    Note: This function is largely AI-generated, with manual edits.
      - Adds a left border to the first subcolumn of each top-level group
        in the BODY (data cells),
      - Adds the same border to the HEADER cells for both level 0 and level 1
        at the same positions (using .col{i} classes).
    """
    # Get data, check applicability
    df = styler.data
    if not isinstance(df.columns, pd.MultiIndex) or df.columns.nlevels < 2:
        raise ValueError("Expected MultiIndex columns with ≥2 levels.")

    # Group boundaries (indices of first subcolumn of each new top-level label)
    lvl0 = df.columns.get_level_values(0).to_numpy()
    breaks = np.flatnonzero(lvl0[1:] != lvl0[:-1]) + 1
    breaks = np.r_[0, breaks]  # Include first element

    # Define common styles
    border_left = f"{width} solid {colour}"

    # 1) BODY: draw the vertical rule down through the data
    for i in breaks:
        styler = styler.set_properties(
            subset=pd.IndexSlice[:, df.columns[i]],
            **{"border-left": border_left}
        )

    # 2) HEADER: mirror the exact same left border on header cells
    # Each header cell at position i has classes: th.col_heading.level{0|1}.col{i}
    header_rules = []
    for i in breaks:
        header_rules.extend([
            {"selector": f"th.col_heading.level0.col{i}",
             "props": [("border-left", border_left)]}
            ,
            {"selector": f"th.col_heading.level1.col{i}",
             "props": [("border-left", border_left)]}
        ])

    # Optional: normalize other header borders so only our verticals stand out
    header_rules.extend([
        {"selector": "th.col_heading", "props": [("border-bottom", "0")]},
        # keep overall layout tight and consistent
        {"selector": "table",
         "props": [("border-collapse", "separate"), ("border-spacing", "0")]}
    ])

    styler = styler.set_table_styles((styler.table_styles or []) + header_rules)

    return styler

# Main function: Display statistics
def display_monthly_statistics(comparison_stats: pd.DataFrame, *,
                               label1: str="ERA5–Drought", label2: str="Reproduced", title_suffix: Optional[str]="") -> pdStyler:
    """
    Display the statistics calculated in comparison_monthly_statistics with more style.
    Note: does NOT do the actual calculations, unlike in other notebooks.
    """
    # Use words for month names; cannot be done in styler
    comparison_stats = comparison_stats.rename(MONTHS_NAMED).rename_axis(None, axis=0)

    # Apply styles:
    # Number of digits
    # Caption
    # No sticky index – does not play nice with jupyter-book
    # Apply vertical lines to separate columns
    formatted = comparison_stats.style \
                                .format(precision=4)  \
                                .set_caption(f"{label2} – {label1}{title_suffix}")  \
                                .pipe(_add_matching_vertical_separators)
    
    # Center headers ; AI-generated
    formatted = formatted.set_table_styles(
        (formatted.table_styles or []) + [
            {'selector': 'th.col_heading',
             'props': [('text-align', 'center'),
                       ('vertical-align', 'bottom'),
                      ],
             },
        ]
    )

    return formatted

##### Visualisation

The following cell defines [earthkit-plots styles](https://earthkit-plots.readthedocs.io/en/latest/_api/plots/styles/index.html) for the variables in the datasets.
These styles define the colour maps and colour bar ranges for each quantity. Earthkit-plots styles are explained further in the [corresponding documentation](https://earthkit-plots.readthedocs.io/en/latest/examples/examples/examples.html#Styles).

In [ ]:
# Helper function: Translate tabulated categories to Style
def categories_to_earthkit_style(var_categories: Iterable[Iterable]) -> Style:  # e.g. CATEGORIES_SPI
    """ From a list of categories, create an earthkit.plots Style. """
    cmap = ListedColormap([category[3] for category in var_categories[::-1]])  # Colours in reverse (ascending) order
    norm =   BoundaryNorm([category[0] for category in var_categories[-2::-1]], cmap.N, extend="both")
    style = {"cmap": cmap, "norm": norm, "extend": "both"}
    return style

# Styles for SPI, SPEI based on Keune+25 colours
_style_spi  = categories_to_earthkit_style(CATEGORIES_SPI)
_style_spei = categories_to_earthkit_style(CATEGORIES_SPEI)
_style_spi_diff  = {"cmap": plt.cm.RdBu.resampled(11), "vmin": -1, "vmax": 1, "extend": "both"}
_style_spei_diff = _style_spi_diff

# Styles for quality flags
_style_probability = {"cmap": plt.cm.magma.resampled(30), "vmin": 0, "vmax": 1}
_style_probability_diff = {"cmap": plt.cm.RdBu.resampled(15), "vmin": -1, "vmax": 1}
_style_flag_summed = {"cmap": ListedColormap(["#000000", "#ffffcd", "#f4eabb", "#ebd4ac", "#e2bc99", "#d5a888", "#c8947a",
                                                         "#bf7e6b", "#b4685d", "#a7544e", "#9b3d3f", "#8d2733", "#800029",]),
                      "norm": BoundaryNorm(np.arange(-0.5, 13, 1), 13),
                      "ticks": np.arange(0, 13, 1),
                     }

# Individual styles
# Set up like this so they can still be edited individually
styles = {
    # Indexes
    "SPI":  Style(**_style_spi),  "SPI_diff":  Style(**_style_spi_diff),
    "SPEI": Style(**_style_spei), "SPEI_diff": Style(**_style_spei_diff),

    # Quality flags
    "probability": Style(**_style_probability), "probability_diff": Style(**_style_probability_diff),
    "flag_summed": Style(**_style_flag_summed),
}

# Apply general settings
for style in styles.values():
    style.normalize = False

The following cell contains some base helper functions (e.g. displaying in Jupyter Notebook or Jupyter Book style, adding textboxes with consistent formatting, etc.):

In [ ]:
# Visualisation: Helper functions, general
def _glue_or_show(fig, glue_label=None):
    try:
        glue(glue_label, fig, display=False)
    except TypeError:
        plt.show()
    finally:
        plt.close()

def _add_textbox_to_subplots(text: str, *axs: plt.Axes | ekp.Subplot, right=False) -> None:
    """ Add a text box to each of the specified subplots. """
    # Get the plt.Axes for each ekp.Subplot
    axs = [subplot.ax if isinstance(subplot, ekp.Subplot) else subplot for subplot in axs]

    # Set up location
    x = 0.95 if right else 0.05
    horizontalalignment = "right" if right else "left"

    # Add the text
    for ax in axs:
        ax.text(x, 0.95, text, transform=ax.transAxes,
        horizontalalignment=horizontalalignment, verticalalignment="top",
        bbox={"facecolor": "white", "edgecolor": "black", "boxstyle": "round",
              "alpha": 1})

def plot_zero_line(*axs: plt.Axes) -> None:
    """ Plot the y=0 line with consistent styling. """
    for ax in axs:
        ax.axhline(0, color="black", zorder=1.9)

The following functions are also base helper functions, but specific to geospatial plots:

In [ ]:
# Default geospatial domain
# To do: Use Robinson CRS but maintain aspect ratio
DOMAIN_GLOBAL = ekp.geo.domains.Domain.from_string("global")
# DOMAIN_GLOBAL = ekp.geo.domains.Domain.from_string("global", crs=ccrs.Robinson())

# Visualisation: Helper functions for geospatial plots
def _spatial_plot_append_subplots(fig: ekp.Figure, *data: xr.Dataset, domain: Optional[AnyDomain]=None, **kwargs) -> list[ekp.Subplot]:
    """ Plot any number of datasets into new subplots in an existing earthkit figure. """
    # Create subplots
    subplots = [fig.add_map(domain=domain) for d in data]

    # Plot
    for subplot, d in zip(subplots, data):
        subplot.grid_cells(d, **kwargs)

    return subplots

def decorate_fig(fig: ekp.Figure, *, title: Optional[str]="") -> None:
    """ Decorate an earthkit figure with land, coastlines, etc. """
    # Add progress bar because individual steps can be very slow for large plots
    with tqdm(total=5, desc="Decorating", leave=False) as progressbar:
        fig.land()
        progressbar.update()
        fig.coastlines()
        progressbar.update()
        fig.borders()
        progressbar.update()
        fig.gridlines(linestyle=plt.rcParams["grid.linestyle"])
        progressbar.update()
        fig.title(title)
        progressbar.update()

The following functions handle visualisation of accumulated variables such as total precipitation and water balance:

In [ ]:
# Main function: Plot any accumulated variable
def plot_accumulated_variable(data: xr.Dataset, site: dict[str, slice], var: str, *,
                              accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS,
                              start_at_zero=False, var_label: Optional[str]=None,
                              glue_label: Optional[str]=None) -> None:
    """ Plot accumulated time series for variable `var` in multiple accumulation periods, in one `site`. """
    # Select data in site
    data_site = data.sel(**site)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 6), constrained_layout=True)
    
    # Plot data for each accumulation period
    for period in accumulation_periods:
        period_var = f"{var}{period}"
        data_site[period_var].plot(ax=ax, label=f"{period:2d} months")
    
    # Decorate figure
    ax.set_title(f"{var_label} at ({site['lat']} °N, {site['lon']} °E) accumulated over different periods")
    ax.set_xlabel("Time")
    ax.set_ylabel(f"Accumulated {var_label}")
    ax.legend(title="Accumulation\nperiod", reverse=True, loc="best")
    if start_at_zero:  # Start y-axis at 0, e.g. for total precipitation
        ax.set_ylim(ymin=0)
    else:              # If values can be + or -, show the 0-line clearly
        plot_zero_line(ax)      

    # Show result
    _glue_or_show(fig, glue_label)


# Plot tp, wb with preset names
plot_accumulated_precipitation = partial(plot_accumulated_variable, var="tp", start_at_zero=True,  var_label="Total precipitation [mm]")
plot_accumulated_waterbalance  = partial(plot_accumulated_variable, var="wb", start_at_zero=False, var_label="Water balance [mm]")

The following functions produce time series comparisons between datasets, such as reproduced vs. ERA5–Drought SPI:

In [ ]:
# Helper function: Display SPI / SPEI categories
def plot_index_categories(ax: plt.Axes, categories: Iterable[Iterable]) -> None:
    """ Display SPI / SPEI categories (e.g. "extreme drought") on an Axes panel. """
    for low, high, label, colour in categories:
        ax.axhspan(low, high, facecolor=colour, edgecolor=None, alpha=0.25)


# Main function: Plot time series of absolute values (left) and differences (right)
def plot_time_series_comparison(data1: xr.Dataset, data2: xr.Dataset, var: str, *,
                                accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS,
                                var_categories: Optional[Iterable[Iterable]]=None,  # e.g. CATEGORIES_SPI
                                label1: str="ERA5–Drought", label2: str="Reproduced", title_suffix: Optional[str]="",
                                glue_label: Optional[str]=None) -> None:
    """
    Plot time series of one variable (e.g. SPI) in two datasets.
    Left column: Overlapping time series.
    Right column: Differences (data2 – data1).

    One row for each accumulation period.
    `var` and `accumulation_periods` are provided manually, rather than inferred,
        to allow plotting of individual time series.
    """
    # Calculate difference
    difference = data2 - data1
    difference_label = f"{label2} – {label1}"

    # Setup: Figure
    ncols = 2
    nrows = len(accumulation_periods)  # Note: not checking if accumulation periods are actually present in data1, data2
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey="col", squeeze=False,
                            layout="constrained", gridspec_kw={"wspace": 0.07}, figsize=(10, 2.7*nrows))

    # Setup: Styles for line plots
    timeseries_kwargs = [{"color": "tab:blue", "linestyle": "-", "label": label1},
                         {"color": "tab:orange", "linestyle": "dotted", "label": label2},
                        ]

    # Plot data
    for ax_row, period in zip(axs, accumulation_periods):
        var_period = f"{var}{period}"

        # Plot absolute time series
        for data, style in zip([data1, data2], timeseries_kwargs):
            data[var_period].plot(ax=ax_row[0], **style, zorder=2)

        # Display categories and 0 line
        plot_index_categories(ax_row[0], var_categories)
        plot_zero_line(*ax_row)

        # Plot difference
        difference[var_period].plot(ax=ax_row[1], color="#004488", zorder=2)
        md  = nanmedian(ravel(              difference[var_period]),  axis=0).compute()
        mad = nanmedian(ravel(xr.ufuncs.abs(difference[var_period])), axis=0).compute()
        _add_textbox_to_subplots(f"Median Δ: {md:+.4f}\nMedian |Δ|: {mad:.4f}", ax_row[1], right=True)
        

    # Decorate figure
    for ax in axs.ravel():
        ax.set_title(None)
    axs[0, 0].set_title(f"{var} values")
    axs[0, 1].set_title(f"Difference ({difference_label})")

    axs[0, 0].set_ylim(-8, 8)
    axs[0, 1].set_ylim(-1, 1)

    for ax in axs[:, 0]:
        ax.legend(loc="upper right")

    for ax in axs[:-1].ravel():
        ax.set_xlabel(None)
    for ax in axs[-1]:
        ax.set_xlabel("Time")
    for ax_row, period in zip(axs, accumulation_periods):
        ax_row[0].set_ylabel(f"{var}-{period}")
        ax_row[1].set_ylabel(f"{var}-{period} difference")

    fig.suptitle(f"{label1} vs. {label2}\nTime series for {var}{title_suffix}")
    fig.align_ylabels()

    # Show result
    _glue_or_show(fig, glue_label)


# Plot SPI, SPEI with preset names
plot_time_series_comparison_spi  = partial(plot_time_series_comparison, var="SPI",  var_categories=CATEGORIES_SPI)
plot_time_series_comparison_spei = partial(plot_time_series_comparison, var="SPEI", var_categories=CATEGORIES_SPEI)

The following functions produce confusion matrices for the SPI / SPEI categories, e.g. how many "extremely dry" points in the reproduced dataset are also classified as "extremely dry" in ERA5–Drought:

In [ ]:
# Main function: Plot confusion matrices with counts (text) and fractions (colour)
def plot_confusion_matrices(data1: xr.Dataset, data2: xr.Dataset, var: str, var_categories: Iterable[Iterable], *,  # e.g. CATEGORIES_SPI
                            label1: str="ERA5–Drought", label2: str="Reproduced", title_suffix: Optional[str]="",
                            glue_label: Optional[str]=None) -> None:
    """
    Plot confusion matrices of categories for one variable (e.g. SPI "extremely dry") in two datasets.
    One panel for each data_variable that occurs in both datasets and matches the variable `var`,
        e.g. different accumulation periods.
    """
    # Align datasets
    data1, data2 = align_data_variables(data1, data2, var=var)  # Ensure only matching variables
    data1, data2 = xr.align(data1, data2)  # Ensure only matching points
    # data1 = data1.transpose(*[dim for dim in data2.dims])  # Ensure same order of dimensions
    data1, data2 = [data.stack(point=data2.dims) for data in (data1, data2)]  # Make 1D list

    # Apply categorisation
    categories1, categories2 = [categorise_dataset(data, var_categories) for data in (data1, data2)]

    # Create figure
    n_categories = len(var_categories)
    category_index = np.arange(n_categories)
    n_datavars = len(data2.data_vars)
    ncols = 2
    nrows = (n_datavars // ncols) + 1
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5*ncols, 4*nrows),
                            gridspec_kw={"hspace": 0.01, "wspace": 0.01}, layout="constrained")

    # Loop over data variables
    for ax, data_var in zip(axs.ravel(), data2.data_vars):
        # Create confusion matrix from categorisation values
        matrix_absolute = confusion_matrix(categories1[data_var], categories2[data_var], labels=category_index+1)
        matrix_relative = confusion_matrix(categories1[data_var], categories2[data_var], labels=category_index+1, normalize="true")  # Normalised by rows (data1)
        # For operational code, you would add an assert here to check that the sum of elements in matrix_absolute
        # corresponds to the number of not-NaN elements of the input data

        # Plot fractions for color background
        image = ax.imshow(matrix_relative, cmap=plt.cm.Blues.resampled(10), vmin=0.0, vmax=1.0)

        # Annotate with absolute counts
        for i in category_index:
            for j in category_index:
                counts_abs, counts_rel = matrix_absolute[i, j], matrix_relative[i, j]
                ax.text(j, i, f"{counts_abs:.0f}", ha="center", va="center",  # Add text
                        color="white" if counts_rel > 0.5 else "black",       # Colour based on background
                        fontsize=9, clip_on=True)                             # Text settings

        # Label based on data variable, adding a hyphen
        data_var_hyphenated = data_var.replace(var, var+"-")
        ax.set_title(data_var_hyphenated)

    # Add category labels
    category_labels = [category[2] for category in var_categories]
    for ax in axs.ravel():
        ax.set_xticks(category_index, labels=category_labels, rotation=30, ha="right", fontsize=9)
        ax.set_yticks(category_index, labels=category_labels, fontsize=9)
        ax.grid(False)
        ax.xaxis.set_inverted(True)

    # Add a single shared colorbar showing row-wise fraction
    cbar = fig.colorbar(image, ax=axs, location="right", fraction=0.05, pad=0.05)# shrink=0.6)
    cbar.set_label(f"Fraction of {label1} data (rows)")

    # Hide extra panels if odd number of variables
    for ax in axs.ravel()[n_datavars:]:
        ax.set_visible(False)

    # Decorate figure
    fig.suptitle(f"{label1} vs. {label2}\nConfusion matrices for {var}{title_suffix}")
    fig.supxlabel(label2, fontweight="bold", fontsize=16)
    fig.supylabel(label1, fontweight="bold", fontsize=16)

    # Show result
    _glue_or_show(fig, glue_label)


# Plot SPI, SPEI with preset names
plot_confusion_matrices_spi  = partial(plot_confusion_matrices, var="SPI",  var_categories=CATEGORIES_SPI)
plot_confusion_matrices_spei = partial(plot_confusion_matrices, var="SPEI", var_categories=CATEGORIES_SPEI)

The following functions display geospatial comparisons between index values:

In [ ]:
# Main function: Plot maps of absolute values (left, middle) and differences (right)
def plot_geospatial_comparison(data1: xr.Dataset, data2: xr.Dataset, var: str, time: str, *,
                               label1: str="ERA5–Drought", label2: str="Reproduced", title_suffix: Optional[str]="",
                               domain: AnyDomain=DOMAIN_GLOBAL,
                               time_dimension: str="time",
                               glue_label: Optional[str]=None) -> None:
    """
    Plot geospatial views of categories for one variable (e.g. SPI "extremely dry") in two datasets.
    One panel for each data_variable that occurs in both datasets and matches the variable `var`,
        e.g. different accumulation periods.
    """
    # Select data for one timestamp
    data1, data2 = [data.sel({time_dimension: time})
                    for data in (data1, data2)]

    # Only look at variables that are in both datasets
    # Note: Not checking for matching coordinates / xr.align
    data1, data2 = align_data_variables(data1, data2, var=var)

    # Calculate difference
    difference = data2 - data1
    difference_label = f"{label2} – {label1}"

    # Setup: Figure
    nrows = len(data2.data_vars)
    ncols = 3  # data1, data2, diff
    fig = ekp.Figure(rows=nrows, columns=ncols, size=(4*ncols, 3.7*nrows))

    # Loop over variables (e.g. accumulation periods)
    for data_var in data2.data_vars:
        # Plot index
        for data in (data1, data2):
            subplot = fig.add_map(domain=domain)
            subplot.grid_cells(data, z=data_var, style=styles[var])

        # Plot difference
        subplot_diff = fig.add_map(domain=domain)
        subplot_diff.grid_cells(difference, z=data_var, style=styles[var+"_diff"])

    # Label dataset at the top
    for subplot, label in zip(fig.subplots[:ncols], [label1, label2, difference_label]):
        subplot.title(label)

    # Label indicator in the corner
    for subplots, data_var in zip(batched(fig.subplots, ncols), data2.data_vars):
        _add_textbox_to_subplots(data_var, *subplots)

    # Legend at the bottom
    for subplot in fig.subplots[-ncols:]:
        subplot.legend(label=var)

    # Decorate figure
    time_for_title = time[:7] # Could be changed to proper datetime format, e.g. %B %Y
    title = f"{label1} vs. {label2}\nGeospatial comparison for {var} ({domain.name}, {time_for_title}){title_suffix}"
    decorate_fig(fig, title=title)

    # Show result
    _glue_or_show(fig.fig, glue_label)

The following functions display geospatial comparisons between quality flags:

In [ ]:
# Main function: Plot maps of absolute values (left, middle) and differences (right)
def plot_geospatial_comparison_quality_flags(probability1: xr.Dataset, probability2: xr.Dataset, var: str, example_month: int, *,
                                             masks: Optional[dict[str, Iterable[xr.Dataset]]]=None,  # e.g. p0 < 0.33 or alpha > 0.05
                                             label1: str="ERA5–Drought", label2: str="Reproduced", title_suffix: Optional[str]="",
                                             flag_label: Optional[str]=None,
                                             domain: AnyDomain=DOMAIN_GLOBAL, shared_mask: Optional[xr.DataArray]=True,
                                             month_dimension: str="month",
                                             glue_label: Optional[str]=None) -> None:
    """
    Plot geospatial views of pre-calculated quality flags in two datasets.
    probability1, probability2 are datasets with quality flag probabilities.
        These are displayed for variable `var` (e.g. tp3, SPEI3) in `example_month` (e.g. 12).
    Additional masks created from the quality flags can be included, e.g. where p0 <= 0.33.
        These are ingested as a dict with str keys (the label for that mask) and an Iterable of xr.Datasets,
        corresponding to the mask in probability1 and in probability2.
        Following Keune+25 Fig 3, masks are displayed as their sum over all 12 months (left, middle)
        as well as the number of mismatched months (right).
    An additional `shared_mask`, e.g. a land-sea mask, can be applied to all panels.
    """
    # Select data for one month
    probability1, probability2 = [prob.sel({month_dimension: example_month})
                                  for prob in (probability1, probability2)]

    # Calculate difference
    probability_difference = probability2 - probability1
    difference_label = f"{label2} – {label1}"

    # Setup: Figure
    nrows = 1 + len(masks)  # 1 for probabilities + 1 each per mask
    ncols = 3  # data1, data2, diff
    figsize = (4*ncols, 3*nrows) if domain.name == "Global" else (4*ncols, 3.7*nrows)  # Adjust aspect ratio for global plots
    fig = ekp.Figure(rows=nrows, columns=ncols, size=figsize)

    # Plot probabilities
    for prob in (probability1, probability2):
        subplot = fig.add_map(domain=domain)
        subplot.grid_cells(prob.where(shared_mask), z=var, style=styles["probability"])

    subplot = fig.add_map(domain=domain)
    subplot.grid_cells(probability_difference.where(shared_mask), z=var, style=styles["probability_diff"])

    # Plot masks (if any)
    for key, list_of_datasets in masks.items():
        # Plot masks
        for mask in list_of_datasets:
            # Add up per month and invert (to get number of *flagged* rather than *allowed* points)
            flagged_months = count_flagged_months(mask)

            # Plot
            subplot = fig.add_map(domain=domain)
            subplot.grid_cells(flagged_months.where(shared_mask), z=var, style=styles["flag_summed"])

        # Find mismatched months
        mask1, mask2 = list_of_datasets
        masks_not_matching = (mask1 != mask2)
        months_not_matching = count_flagged_months(masks_not_matching, invert=False)

        # Plot number of mismatched months
        subplot = fig.add_map(domain=domain)
        subplot.grid_cells(months_not_matching.where(shared_mask), z=var, style=styles["flag_summed"])

    # Label dataset at the top
    for subplot, label in zip(fig.subplots[:ncols], [label1, label2, difference_label]):
        subplot.title(label)

    # Label masks in the corner
    for subplots, key in zip(batched(fig.subplots[ncols:], ncols), masks.keys()):
        _add_textbox_to_subplots(key, *subplots)

    # Legend at the sides
    fig.subplots[0].legend(label=flag_label, location="left")
    fig.subplots[ncols-1].legend(label="Difference", location="right")
    for subplot in fig.subplots[ncols::ncols]:
        subplot.legend(label="Flagged months", location="left")
    for subplot in fig.subplots[2*ncols-1::ncols]:
        subplot.legend(label="Mismatched months", location="right")

    # Decorate figure
    month_for_title = MONTHS_NAMED[example_month]
    title = f"{label1} vs. {label2}\nGeospatial comparison of {flag_label}\n({domain.name}, {month_for_title}, {var}){title_suffix}"
    decorate_fig(fig, title=title)

    # Show result
    _glue_or_show(fig.fig, glue_label)

(section-general_setup)=
### 2. General setup
This section provides some of the setup for the further analysis,
including
the timespan and sites to investigate
as well as
the CDS data downloads.
This ensures that the following sections
(SPI, SPEI, etc.)
can be run independently.

#### 2.1 Analysis setup
In this assessment,
we will calculate SPI and SPEI for the years 1940–2024.
For the reference period,
we will use the World Meteorological Organization (WMO) current standard 30-year reference period of 1991–2020,
which is also used in ERA5-Drought.
Both of these date ranges can be adjusted in the cell below when running the analysis yourself:

In [ ]:
# Define your preferred analysis and reference periods
years           = (1940, 2024)  # Years for the analysis (inclusive)
years_reference = (1991, 2020)  # Years for the reference period (inclusive)

# Derived variables for convenience:
reference_window = {"time": slice(f"{years_reference[0]}-01-01", f"{years_reference[1]}-12-01"),}  #  Slice (1991-01-01, 2020-12-01)
entire_window    = {"time": slice(f"{years[0]}-01-01",           f"{years[1]}-12-01"),}            #  Slice (1940-01-01, 2024-12-01)

Some parts of the analysis will be performed globally, others in specific sites for computational reasons.
This notebook uses Addis Ababa in Ethiopia (9.00 °N, 38.75 °E) and the Horn of Africa region around it as an example;
a different site can be chosen when running the notebook yourself by editing the cell below:

In [ ]:
# Define your preferred site for the time series analysis
example_site = {"lat": 9.00, "lon": 38.75}  # Compatible with ERA5–Drought and pre-processed ERA5 data

# Define the size and name of the surrounding region
example_region_size = 12  # Degrees to either side of the example side
example_region_size = 2  # TESTING ONLY
label_region = "Horn of Africa"

# Define the year for the example region analysis
snapshot_year = 2024

# Derived variables for convenience
# Example site
label_site = f"({example_site['lat']:.2f} °N, {example_site['lon']:.2f} °E)"
request_site = request_data_for_one_site(lat=example_site["lat"], lon=example_site["lon"])

# Example region
example_region = {"lat": slice(example_site["lat"] + example_region_size, example_site["lat"] - example_region_size),  # Decreasing order
                  "lon": slice(example_site["lon"] - example_region_size, example_site["lon"] + example_region_size),} # Increasing order
request_region = request_data_for_one_site(lat=example_site["lat"], lon=example_site["lon"], half_width=example_region_size)
domain_region = ekp.geo.domains.Domain(
    bbox=[example_region["lon"].start, example_region["lon"].stop, example_region["lat"].start, example_region["lat"].stop],
    name=label_region,
)

#### 2.2 CDS download setup
Having defined our target years, we can now define our CDS request.
First, we define templates with some default parameters
(e.g. years, data format)
that will also be used later in the notebook.
Additional information for specific downloads
(e.g. variable, data stream)
is mixed into this template where relevant.

This notebook uses [earthkit-data](https://github.com/ecmwf/earthkit-data) to download files from the CDS.
If you intend to run this notebook multiple times, it is highly recommended that you [enable caching](https://earthkit-data.readthedocs.io/en/latest/guide/caching.html) to prevent having to download the same files multiple times.
If you prefer not to use earthkit, the following requests can also be used with the [cdsapi module](https://cds.climate.copernicus.eu/how-to-api#linux-use-client-step).
In either case (earthkit-data or cdsapi), it is required to set up a CDS account and API key as explained [on the CDS website](https://cds.climate.copernicus.eu/how-to-api).

##### ERA5
We start by setting up a template for requests from the [_Complete ERA5 global atmospheric reanalysis_](https://doi.org/10.24381/cds.143582cf) dataset, from which we will obtain precipitation and (for SPEI) potential evapotranspiration:

In [ ]:
ID_ERA5 = "reanalysis-era5-complete"

request_era5_template = {
    "class": "ea",            # Default for ERA5
    # Dates: ERA5 takes these in the format 19400101/19400201/.../20241101/20241201
    # The following lines generate a string in said format for the chosen year range
    "date": "/".join(f"{year}{month:02}01"                    # yyyymm01 format
                     for year in range(years[0], years[1]+1)  # All years in specified range, inclusive
                     for month in MONTHS),                    # All calendar months
    "expver": "1",            # ERA5 consolidated data
    "levtype": "sfc",         # Surface
    "grid": "0.25/0.25",      # Grid: 0.25° by 0.25° (interpolated from native)
    "type": "fc",             # Forecast
    "data_format": "netcdf",  # NetCDF data
}

##### ERA5–Drought
We also set up a template for requests from ERA5–Drought ([_Monthly drought indices from 1940 to present derived from ERA5 reanalysis_](https://doi.org/10.24381/9bea5e16)):

In [ ]:
ID_ERA5_DROUGHT = "derived-drought-historical-monthly"

# General request template
request_era5drought_template = {
    "version": "1_0",                                # Current version
    "dataset_type": "consolidated_dataset",          # Only use consolidated data, i.e. not ERA5T-derived
    "month": [f"{month:02d}" for month in MONTHS],   # All calendar months
}

# Request for SPI or SPEI data
request_era5drought_index = {
    "product_type": ["reanalysis"],
    "accumulation_period": [str(p) for p in ACCUMULATION_PERIODS],
    "year": [f"{year}" for year in range(years[0], years[1]+1)],    
} | request_era5drought_template

# Request for quality flags
request_era5drought_flag = {
    "product_type": ["reanalysis"],
} | request_era5drought_template

#### 2.3 Download land-sea mask
ERA5–Drought uses a land-sea mask to only provide values over land.
Here, we download the [ERA5 land-sea mask](https://confluence.ecmwf.int/display/FUG/Section+2.1.3.1+Land-Sea+Mask) from the [_ERA5 hourly data on single levels from 1940 to present_](https://doi.org/10.24381/cds.adbb2d47) dataset:

In [ ]:
ID_ERA5_LANDSEAMASK = "reanalysis-era5-single-levels"

request_landsea_mask = {
    "product_type": "reanalysis",
    "variable": "land_sea_mask",
    "date": "2000-01-01",  # Does nothing but is required in CDS form
    "time": "00:00",       # Does nothing but is required in CDS form
    "format": "netcdf",
    "download_format": "zip",
}

# Download mask
LAND = ekd.from_source("cds", ID_ERA5_LANDSEAMASK, request_landsea_mask)
LAND = LAND.to_xarray(compat="equals")  # Convert to xarray dataset

# Convert longitude from 0..360 to –180..180
LAND = rename_era5_dimensions(LAND)
LAND = longitude_360_to_180(LAND, "lon")

# Drop time dimension
LAND = LAND.squeeze("time")

# Convert to boolean mask: Land if > 0.5 (following ERA5 convention)
LAND = (LAND["lsm"] > 0.5) # Land considered to be full

(section-spi)=
### 3. SPI comparison

#### 3.1 Download monthly precipitation data from ERA5
First, the monthly-mean total precipitation data from the ERA5 reanalysis is downloaded.
Generally, one would use the [_ERA5 monthly averaged data on single levels from 1940 to present_](https://doi.org/10.24381/cds.f17050d7) dataset for this, which provides pre-calculated monthly means at 0.25° by 0.25° resolution.
For this assessment,
to be as close to the ERA5–Drought data processing pipeline as possible
and
to make use of some of MARS's functionalities (see [](section-ensemble)),
we instead use the [_Complete ERA5 global atmospheric reanalysis_](https://doi.org/10.24381/cds.143582cf) dataset.

We want to download 
total precipitation data (variable `228.128`)
from the
`moda` stream (monthly-mean reanalysis data),
so we mix this information into the request template set up [previously](section-general_setup)
and submit the request to the CDS.
More information about the format for these requests is available in the [MARS ERA5 catalogue](https://apps.ecmwf.int/data-catalogues/era5/?class=ea).

In [ ]:
request_era5_precipitation_moda = {
    "param": "228.128",  # Variable: Total precipitation
    "stream": "moda",    # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and load into xarray
data_era5_precipitation_cds = ekd.from_source("cds", ID_ERA5, request_era5_precipitation_moda)  # Download as field list
data_era5_precipitation_cds = data_era5_precipitation_cds.to_xarray(compat="equals")  # Convert to xarray dataset

# Pre-process to desired format
# Note the change in variable name
data_era5_precipitation_preprocessed = preprocess_era5(data_era5_precipitation_cds)

# Display in notebook
data_era5_precipitation_preprocessed

#### 3.2 Calculate total precipitation over accumulation periods
As explained above,
SPI and SPEI are commonly evaluated over different accumulation periods.
ERA5–Drought provides both indices for periods of 1, 3, 6, 12, 24, 36, and 48 months.
Here, we perform this accumulation by calculating the total precipitation over the previous _p_ months
at every coordinate and timestamp in the ERA5 data,
for each accumulation period _p_.
The implementation used here
(which can be found in [](section-code_setup))
accounts for the variable number of days in each month,
including leap years.

The resulting accumulated time series for the example site defined in [](section-general_setup),
Addis Ababa in Ethiopia in our example,
are displayed in Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-tp-accumulated>`.
This example clearly shows how shorter accumulation periods probe short-term effects such as seasonality,
which are smoothed out in longer periods.

In [ ]:
# Accumulate total precipitation
data_era5_precipitation = accumulate(data_era5_precipitation_preprocessed, "tp")

# Display result
data_era5_precipitation

In [ ]:
# Display accumulated precipitation at example site
plot_accumulated_precipitation(data_era5_precipitation, example_site,
                               glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-tp-accumulated")

```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-tp-accumulated
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-tp-accumulated"

Total precipitation from ERA5 accumulated over different periods, for the example site of Addis Ababa, Ethiopia.
```

#### 3.3 Fit gamma distribution by calendar month
At the core of SPI is the assumption that (monthly) total precipitation values follow a gamma distribution.
Here, this distribution is fitted to data within the reference period (1991–2020 by default, see above).
A separate distribution is fitted for each calendar month,
i.e.
we fit one distribution for all 30 Januaries in the reference period,
another for all 30 Februaries,
and so on.
This separation allows the resulting index to account for seasonal differences.
This is then repeated for each accumulation period.

Here, we use
[scipy.stats.gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html)
to fit the gamma distribution.
This returns three parameters for each calendar month and accumulation period, namely
shape (_α_), location (_μ_), and scale (_β_).

In [ ]:
# Select data in reference window
data_era5_precipitation_reference = data_era5_precipitation.sel(**reference_window)

# Fit gamma distribution
spi_parameters = fit_monthly_spi(data_era5_precipitation_reference)

# Display result
spi_parameters

The observed (accumulated) total precipitation values along the entire time series
(1940–2024 in our example)
are then compared to the fitted parameters.
This is quantified in terms of where the observed values fall on the cumulative distribution function (CDF):

$$
P(X \leq x) = F_X(x ; \alpha, \mu, \beta) = \int_{-\infty}^{x}f(y ; \alpha, \mu, \beta) \, \text{d}y
$$

Where
_P(X ≤ x)_ is the probability for a randomly drawn total precipitation _X_ to be equal to or less than the observed value _x_;
_F{sub}`X`_ is the CDF for a gamma distribution with shape _α_, location _µ_, and scale _β_;
and
the CDF is equal to the integral of the corresponding probability density function up to _x_.

As before, there is a distribution for each calendar month, for each accumulation period.

Note that actually evaluating the CDF
– as opposed to [queueing it up in dask](https://docs.xarray.dev/en/stable/user-guide/dask.html), as done here –
can be slow, especially for a large dataset like global ERA5 precipitation.
As such, if you are interested in a subset of the data,
such as a specific site or period in time,
it may be best to subset your data _before_ calculating the CDF rather than afterwards.
An example of this is provided below in the comparison with ERA5–Drought SPI.

In [ ]:
# Compute CDF time series
cdf = compute_cdf_spi(data_era5_precipitation, spi_parameters)

# Display result
cdf

#### 3.4 Adjust for months with zero precipitation
Sometimes, zero (or near-zero) monthly precipitation is observed,
particularly in dry areas like deserts.
Maintaining the desired statistics,
such as the mean SPI in the reference period being 0,
requires adjusting the CDF to account for these zero-precipitation months.

ERA5–Drought follows the [[Stagge+15](https://doi.org/10.1002/joc.4267)] method to adjust the CDF.
First, the probability of zero precipitation is calculated as follows:

$$
\bar{p_0} = \frac{n_0 + 1}{2 (n + 1)}
$$

where
_n{sub}`0`_ is the number of months with zero precipitation in the reference period,
and
_n_ is the total number of months in the reference period (here 30 for 1991–2020).
While some datasets define "zero precipitation" using a slightly higher threshold
(e.g. less than 0.1 mm)
to account for measurement uncertainty,
ERA5–Drought uses a threshold of exactly 0 mm total precipitation in the underpinning ERA5 data.

Having calculated the probability of zero precipitation, the CDF is adjusted as follows:

$$
F'_X(x ; \alpha, \mu, \beta) =
\begin{cases}
\bar{p_0} + (1 - \bar{p_0}) \, F_X(x; \alpha, \mu, \beta), & x > 0, \\
\frac{n_0 + 1}{2(n+1)}, & x = 0
\end{cases}
$$

As before, this adjustment is carried out individually for each calendar month and accumulation period.

In [ ]:
# Adjust CDF
cdf_adjusted = adjust_cdf_for_zero_precipitation(cdf, data_era5_precipitation_reference)

# Display result
cdf_adjusted

#### 3.5 Compute SPI
Finally,
SPI values
for each data point (latitude, longitude, time)
are calculated from the adjusted CDF values _F'{sub}`X`_ by transforming to a standard normal distribution.
The function [scipy.stats.norm.ppf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) is used to calculate the inverse CDF _Φ{sup}`-1`_ of the normal distribution:

$$
\text{SPI}(x) = \Phi^{-1}\left(F'_X(x ; \alpha, \mu, \beta)\right)
$$

In [ ]:
# Calculate SPI from adjusted CDF
spi_reproduced = cdf_to_spi(cdf_adjusted)

# Display result
spi_reproduced

#### 3.6 SPI comparison: Time series in example site
Having reproduced the SPI index from ERA5 precipitation data following the ERA5–Drought methodology,
we can now compare the results to determine the reproducibility of ERA5–Drought.
We first compare the datasets over time at the example site defined in [](section-general_setup).

The previous sections set up the SPI reproduction pipeline for the entire ERA5 precipitation dataset (global, 1940–2024).
Because actually executing said calculation takes a long time,
as noted in the CDF subsection,
here we sub-select the downloaded ERA5 precipitation data and calculate SPI for the example site only.
For convenience, the full SPI pipeline is wrapped into a single `calculate_spi_from_era5` function,
as defined in [](section-code_setup).

Note that while the two may seem equivalent,
downloading the ERA5 precipitation data for the example site only and then running the SPI pipeline,
rather than downloading the entire dataset and subselecting for the example site,
produces different results.
This is likely caused by the inner workings of the MARS regridding function.
For consistency with ERA5–Drought, it is necessary to download the global ERA5 dataset and subselect.

In [ ]:
# Select only desired site
data_era5_precipitation_site = data_era5_precipitation_preprocessed.sel(**example_site)

# Calculate SPI
spi_reproduced_site = calculate_spi_from_era5(data_era5_precipitation_site, reference_window=reference_window)

# Persist in memory to speed up analysis – this step may take a few minutes
spi_reproduced_site = spi_reproduced_site.persist()

# Display result
spi_reproduced_site

Next, we download the corresponding data from ERA5–Drought.
Because of size limits on the CDS,
the time series must be downloaded in parts:

In [ ]:
# Main request, based on template and example site
request_era5drought_spi = {
    "variable": ["standardised_precipitation_index"],
} | request_era5drought_index | request_site

# Split into batches of up to 20 years each
subrequests_era5drought_spi = batch_requests(request_era5drought_spi, n=20)

# Download data and load into xarray
spi_era5drought = ekd.from_source("cds", ID_ERA5_DROUGHT, *subrequests_era5drought_spi)
spi_era5drought = spi_era5drought.to_xarray(compat="equals", join="outer")
spi_era5drought = spi_era5drought.chunk({"time": -1})  # Full time series in one chunk

# Select only desired site (remove margins)
spi_era5drought = spi_era5drought.sel(**example_site)

# Persist in memory to speed up analysis – this step may take a few minutes
spi_era5drought = spi_era5drought.persist()

# Display result
spi_era5drought

First, we examine the per-point difference in SPI between ERA5–Drought and the reproduction.
This comparison is performed separately for each calendar month and each accumulation period
to reflect the fitting process.
We calculate the median difference _Δ_ and median absolute difference _|Δ|_:

In [ ]:
# Calculate SPI median differences
spi_difference_site = comparison_monthly_statistics(spi_era5drought, spi_reproduced_site)

# Display with style
display_monthly_statistics(spi_difference_site)

**Description of comparisons**

Table for quantitative differences

Median and absolute median difference for each accumulation window across months are then calculated  

**TODO point-to-make**: SPI is only discrepant for certain months (but that this discrepancy is still quite small)- since fitting is by month only.

**Description of figures**

**TODO point-to-make**: SPI for location in Addis Ababa shows excellent agreement across entire time series. Showed that our methodology is consistent with that of ERA5 and reproduceable.  
Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-timeseries>`  
Same seen in different sites (not shown here)  

A confusion matrix is generated to compare drought classifications for the single, above location, between the ERA5‑Drought dataset and the reproduced dataset, capturing any discrepancies. The matrix is computed over the full time period (1940–2024), where each count corresponds to a single monthly timestep.  
Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion>`  
**TODO point-to-make**: and that in fact, the indices fall under the same drought severity classification for all timestamps at Addis Ababa.  

In [ ]:
# Plot time series comparison (absolute values and differences)
plot_time_series_comparison_spi(spi_era5drought, spi_reproduced_site, title_suffix=f" at {label_site}",
                                glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-timeseries")

In [ ]:
# Plot confusion matrix comparison
# e.g. are all "extremely dry" data in ERA5–Drought also "extremely dry" in the reproduction?
plot_confusion_matrices_spi(spi_era5drought, spi_reproduced_site, title_suffix=f" at {label_site}",
                            glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion")

::::{tab-set}
:::{tab-item} Time series
:sync: timeseries
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-timeseries
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-timeseries"

SPI time series downloaded from ERA5–Drought and reproduced from ERA5 precipitation data (left) and the difference between the two (right), for the example site of Addis Ababa, Ethiopia.
Colours in the left-hand column correspond to SPI categories (e.g. "extremely dry") in [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)].
```
:::
:::{tab-item} Confusion matrix
:sync: confusion
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion"

Confusion matrices for SPI categories from ERA5–Drought vs. reproduced from ERA5, for the example site of Addis Ababa, Ethiopia, in 1940–2024.
```
:::
::::

#### 3.7 SPI comparison: Regional snapshot
Next, we investigate spatial patterns in SPI and the difference therein across a wider region around the example site.
This region is defined in [](section-general_setup).
In this example, we look at part of the Horn of Africa using a box of 12° in all directions around Addis Ababa, Ethiopia.
To reduce computing requirements, the comparison is performed for one year only,
here 2024, again defined in [](section-general_setup).

As in the time series comparison,
we subselect the desired data from ERA5 and calculate the corresponding SPI
and
download SPI from ERA5–Drought for the desired region and time span:

In [ ]:
# Select only desired site
data_era5_precipitation_region = data_era5_precipitation_preprocessed.sel(**example_region)

# Window in which to evaluate SPI
evaluation_window = {"time": slice(f"{snapshot_year}-01-01", f"{snapshot_year}-12-01")}  #  Slice (2024-01-01, 2024-12-01)

# Calculate SPI
spi_reproduced_region = calculate_spi_from_era5(data_era5_precipitation_region, reference_window=reference_window,
                                                evaluation_window=evaluation_window)

# Persist in memory to speed up analysis – this step may take a few minutes
spi_reproduced_region = spi_reproduced_region.persist()

# Display result
spi_reproduced_region

In [ ]:
# Main request, based on template and example region
request_era5drought_spi = {
    "variable": ["standardised_precipitation_index"],
} | request_era5drought_index | request_region

# Select only desired year
request_era5drought_spi = request_era5drought_spi | {"year": [f"{snapshot_year}"],}

# Download data and load into xarray
spi_era5drought = ekd.from_source("cds", ID_ERA5_DROUGHT, request_era5drought_spi)
spi_era5drought = spi_era5drought.to_xarray(compat="equals", join="outer")
spi_era5drought = spi_era5drought.chunk({"time": -1})  # Full time series in one chunk

# Persist in memory to speed up analysis – this step may take a few minutes
spi_era5drought = spi_era5drought.persist()

# Display result
spi_era5drought

First, we examine the per-point difference in SPI between ERA5–Drought and the reproduction.
This comparison is performed separately for each calendar month and each accumulation period
to reflect the fitting process.
We calculate the median difference _Δ_ and median absolute difference _|Δ|_:

In [ ]:
# Calculate SPI median differences
spi_difference_region = comparison_monthly_statistics(spi_era5drought, spi_reproduced_region)

# Display with style
display_monthly_statistics(spi_difference_region)

Analysis text

(Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-geospatial>`)
(Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-region>`)

In [ ]:
plot_geospatial_comparison(spi_era5drought, spi_reproduced_region, var="SPI", time=f"{snapshot_year}-12-01",
                           domain=domain_region,
                           glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-geospatial")

In [ ]:
# Plot confusion matrix comparison
# e.g. are all "extremely dry" data in ERA5–Drought also "extremely dry" in the reproduction?
plot_confusion_matrices_spi(spi_era5drought, spi_reproduced_region, title_suffix=f" in {label_region}",
                            glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-region")

::::{tab-set}
:::{tab-item} Geospatial
:sync: geospatial
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-geospatial
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-geospatial"

SPI downloaded from ERA5–Drought (left) vs. reproduced from ERA5 precipitation data (middle) and the difference between the two (right), for the region around the example site of Addis Ababa, Ethiopia.
Only one month (December 2024) is displayed.
Colours correspond to SPI categories (e.g. "extremely dry") in [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)].
```
:::
:::{tab-item} Confusion matrix
:sync: confusion
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-region
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-region"

Confusion matrices for SPI categories from ERA5–Drought vs. reproduced from ERA5, across the region around the example site of Addis Ababa, Ethiopia.
Combines data for all months within one year (2024).
```
:::
::::

#### 3.8 Quality flags: Probability of zero precipitation
One of the quality flags included in ERA5–Drought is the probability of zero precipitation _p{sub}`0`_.
This represents,
for each calendar month and accumulation period,
the fraction of months within the reference period (1991–2020) where precipitation was zero.
While the ERA5–Drought implementation of SPI accounts for months with zero precipitation by shifting the CDF,
as demonstrated above,
those months do skew the fitted distribution and reduce the reliability of the index.
In ERA5–Drought,
the gamma distribution is not even fitted if at least 10 out of 30 months in the reference window have 0 precipitation,
and the authors recommend that users filter out locations with a lower threshold, such as _p{sub}`0`_ > 0.1 [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)].


Unlike in the zero-precipitation correction that is part of the SPI calculation,
this quality flag uses an unweighted probability, with _n{sub}`0`_ the number of months with zero precipitation and _n_ the total number of months in the reference window (30):
$$
p_0 = \frac{n_0}{n + 1}
$$

In this subsection, we reproduce the _p{sub}`0`_ quality flag and its derived masks,
and compare their values to those provided in ERA5–Drought.

In [ ]:
# Accumulate total precipitation
data_era5_precipitation = accumulate(data_era5_precipitation_preprocessed, "tp")

# Select data in reference window
data_era5_precipitation_reference = data_era5_precipitation.sel(**reference_window)

# Calculate the probability of zero precipitation
p_zero_reproduced = probability_of_zero_precipitation(data_era5_precipitation_reference)
# p_zero = p_zero.chunk({"lat": -1, "lon": -1})  # Rechunk to global maps for efficiency
# p_zero = p_zero.persist()  # Persist in memory

# Display result
p_zero_reproduced

The CDS provides ERA5–Drought's _p{sub}`0`_ flag for each accumulation period,
but does not distinguish between them in the variable name.
This means that _p{sub}`0`_ must be downloaded separately for each accumulation period, renamed, and merged together,
rather than downloading it for all in one request:

In [ ]:
# Setup: Request for each accumulation period
request_p_zero = {
    "variable": ["probability_of_zero_precipitation_spi"],
} | request_era5drought_flag

requests_per_accumulation_period = {period: {"accumulation_period": [str(period)],}
                                    for period in ACCUMULATION_PERIODS}

subrequests_era5drought_p_zero = {period: (req | request_p_zero)
                                  for period, req in requests_per_accumulation_period.items()}

# Download data in individual requests
p_zero_era5drought = {period: ekd.from_source("cds", ID_ERA5_DROUGHT, subreq)  # Download as field lists
                      for period, subreq in subrequests_era5drought_p_zero.items()}

# Pre-process: Convert to xarray, handle variable names, month dimension
p_zero_era5drought = preprocess_era5drought_qualityflag(p_zero_era5drought, "tp")

# Display in notebook
p_zero_era5drought

For both datasets, we create masks corresponding to
the threshold for not fitting a gamma distribution (_p{sub}`0`_ > 0.33)
and
the recommended threshold for filtering data (_p{sub}`0`_ > 0.1):

In [ ]:
# Create masks from p_zero
# True: below threshold, allowed
# False: above threshold, flagged
p_zero_reproduced_033 = (p_zero_reproduced <= 0.33)
p_zero_era5drought_033 = (p_zero_era5drought <= 0.33)

p_zero_reproduced_010 = (p_zero_reproduced <= 0.10)
p_zero_era5drought_010 = (p_zero_era5drought <= 0.10)

# Display result
p_zero_era5drought_033

**TODO point-to-make**: the zero precipitation quality flag is in fact reproduceable, and with the above, different thresholds can be set by the user (depending on how strict the threshold is). This gives us faith too that the zero precipitation adjustment has been made correctly (pending formula).

(Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-p0-geospatial>`)
(Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-region>`)

In [ ]:
# Package masks into dictionary for plotting function
masks = {r"$p_0 \leq 0.33$": [p_zero_era5drought_033, p_zero_reproduced_033],
         r"$p_0 \leq 0.10$": [p_zero_era5drought_010, p_zero_reproduced_010],
         }

# Display quality flags – this step may take a few minutes
plot_geospatial_comparison_quality_flags(p_zero_era5drought, p_zero_reproduced, "tp3", example_month=12,
                                         masks=masks, shared_mask=LAND,
                                         flag_label=r"$p_0$ (probability of zero precipitation)",
                                         glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-p0-geospatial")

::::{tab-set}
:::{tab-item} Geospatial
:sync: geospatial
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-p0-geospatial
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-p0-geospatial"

Probability of zero precipitation and associated masks in ERA5–Drought (left) vs. reproduced from ERA5 precipitation data (middle) and the difference or mismatch between the two (right).
Probability is displayed for one calendar month (December) across the reference window (1991–2020).
Masks are displayed as the number of calendar months that get flagged, following [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)] Fig. 3.
```
:::
:::{tab-item} Confusion matrix
:sync: confusion
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-x
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion-x"

Confusion matrices for SPI categories from ERA5–Drought vs. reproduced from ERA5, across the region around the example site of Addis Ababa, Ethiopia.
Combines data for all months within one year (2024).
```
:::
::::

We finally show that "probability of zero precipitation months" match for all calendar months at all locations.

In [ ]:
# Calculate statistics
x = np.ravel(p_zero["tp_mm_accum_1m"].where(ls_mask["lsm"]))
y = np.ravel(era5_d_prob_spi_zero_all["prob_zero_1"].where(ls_mask["lsm"]))

p_threshold = 0.33 

x_count = np.count_nonzero(~np.isnan(x))
y_count = np.count_nonzero(~np.isnan(y))

reject_x_count = (x > p_threshold).sum()
reject_y_count = (y > p_threshold).sum()

percent_reject_x = (reject_x_count/x_count)*100
percent_reject_y = (reject_y_count/y_count)*100

In [ ]:
# Create scatter plot
plt.figure(figsize=(8, 6))
hb = plt.scatter(
    x, y,   
)

plt.hlines(y=p_threshold, xmin=0, xmax=p_threshold, colors='g', linestyles='--', linewidth=2)
plt.vlines(x=p_threshold, ymin=0, ymax=p_threshold, colors='g', linestyles='--', linewidth=2)
plt.fill_between([0,p_threshold], 0, p_threshold, facecolor="none", hatch="X", edgecolor="green", linewidth=0.0, label='Acceptance criteria')

# Labels and title
plt.xlabel('Reproduced')
plt.ylabel('ERA5-Drought')
plt.title('Comparison of Probability of Zero Precipitation')
plt.annotate(f"Reproduced rejection freq: {percent_reject_x:.5g}%\n ERA5D rejection freq:{percent_reject_y:.5g}%", xy=(0.5, 0.2))
plt.legend()

# Display plot
plt.show() 
# TODO: Calculate percentage that are the same, ignoring the NaN.

#### 3.9 Quality flags: Shapiro–Wilk normality test
Quality control is performed by the ERA5-Drought team over the entire dataset. This is done by testing if the calculated distribution of the estimated drought indices over the reference period follows a normal distribution with mean 0 and standard deviation 1. 

This test is performed using the Shapiro-Wilks test for normality [[Shapiro+65](https://doi.org/10.1093/biomet/52.3-4.591)], with a $\alpha$ = 0.05 on the data in the reference period (1991-2020). 

If the resultant p-value is less than $\alpha$ = 0.05, the corresponding quality parameter is set to 0 (bad), otherwise set to 1 (good). We perform this test over the calculated SPI values, that are adjusted for zero-precipitation. 

In [ ]:
# Setup: Request for each accumulation period
request_p_normality = {
    "variable": ["test_for_normality_spi"],
} | request_region | request_era5drought_flag

requests_per_accumulation_period = {period: {"accumulation_period": [str(period)],}
                                    for period in ACCUMULATION_PERIODS}

subrequests_era5drought_p_normality = {period: (req | request_p_normality)
                                       for period, req in requests_per_accumulation_period.items()}

# Download data in individual requests
p_normality_era5drought = {period: ekd.from_source("cds", ID_ERA5_DROUGHT, subreq)  # Download as field lists
                           for period, subreq in subrequests_era5drought_p_normality.items()}

# Pre-process: Convert to xarray, handle variable names, month dimension
p_normality_era5drought = preprocess_era5drought_qualityflag(p_normality_era5drought, "SPI")

# Display in notebook
p_normality_era5drought

In [ ]:
# Select latitude and longitude box (40 x 40 here)
dom_lon_min, dom_lon_max = 30, 53  
dom_lat_min, dom_lat_max = -5, 18

lon_min, lon_max = 52, 53  
lat_min, lat_max = 17, 18

# Subset (latitude is descending in ERA5) - should this be wrapped in a function now?
ds_loc = era5_monthly_mean_reanal.sel(
    latitude=slice(lat_max, lat_min),
    longitude=slice(lon_min, lon_max))

# Accumulate precipitations
precipitation_example_global = accumulate(ds_loc, var = "tp") 

# Select reference precipitation window.
precipitation_example_global_reference = precipitation_example_global.sel(**reference_window) 

# Fit distribution to reference window, extract parameters.
params_fitted_global = fit_monthly_spi(precipitation_example_global_reference)

# Calculate CDF  with parameters.
cdf_global = compute_monthly_spi(precipitation_example_global, params_fitted_global)

In [ ]:
# Adjust CDF zero precipitation.
adjusted_cdf_global = zero_precip_adjustment(precipitation_example_global, cdf_global, reference_window)

# Transform adjusted CDF back to SPI
spi_global = cdf_to_znorm_transform(cdf_global, "SPI")
adjusted_spi_global = cdf_to_znorm_transform(adjusted_cdf_global, "SPI")

# Apply land-sea mask over region. 
# adjusted_spi_ds_global = adjusted_spi_global.where(ls_mask["lsm"]) 

In [ ]:
pval_spi_global, calc_quality_spi_global = xr_shapiro_test(spi_global, reference_window, index_name = "SPI")
calc_quality_spi_agg_global = calc_quality_spi_global.sum(dim = "time") # Aggregate across months.

era5_quality_spi_global = era5_drought_api_multiple(indicator = "spi", var = "quality", product_type = "reanalysis")
era5_quality_spi_global_agg = quality_spi_all.sum(dim="time")

accumulation_window = 1

# --- Colormap as you already have ---
beige_to_red = LinearSegmentedColormap.from_list(
    "beige_to_red",
    ["#fbeec2", "#d47a57", "#8b0000"]   # beige → soft red → dark red
)

ZERO_PRECIP_STYLE      = Style(cmap=beige_to_red, vmin=1,  vmax=12,  normalize=False)
BLACK_POINT_STYLE      = Style(cmap=ListedColormap(["black"]), vmin=0, vmax=0.99, normalize=False)
ZERO_PRECIP_DIFF_STYLE = Style(cmap="RdBu_r", vmin=-12, vmax=12, normalize=False)

# ------------------ LEFT PANEL ------------------
da1 = calc_quality_spi_agg_global[f"significance_{accumulation_window}"] # Reproduced
non_ones1 = da1.where(da1 != 0)   # all values except 0
ones1     = da1.where(da1 == 0)   # only values equal to 0

# ------------------ MIDDLE PANEL ------------------
da2 = era5_quality_spi_global_agg[f"significance_{accumulation_window}"].sel(
    lat=slice(lat_max, lat_min),
    lon=slice(lon_min, lon_max)) # ERA5-D

non_ones2 = da2.where(da2 != 0)
ones2     = da2.where(da2 == 0)

In [ ]:
horn = ekp.geo.domains.Domain(
    bbox=[dom_lon_min, dom_lon_max, dom_lat_min, dom_lat_max],
    name="Horn of Africa",
)

# Create figure with 3 columns
fig = ekp.Figure(rows=1, columns=2)

# ---------- Subplot 1 ----------
subplot1 = fig.add_map(domain=horn, row=0, column=0)
mappable1 = subplot1.grid_cells(non_ones1.persist(), style=ZERO_PRECIP_STYLE)
subplot1.grid_cells(ones1.persist(), style=BLACK_POINT_STYLE, zorder=101)
ax1 = subplot1.ax
ax1.set_title(f"Reproduced — # Months S-W  Rejection ({accumulation_window}‑Month Window)")

# ---------- Subplot 2 ----------
subplot2 = fig.add_map(domain=horn, row=0, column=1)
mappable2 = subplot2.grid_cells(non_ones2.persist(), style=ZERO_PRECIP_STYLE)
subplot2.grid_cells(ones2.persist(), style=BLACK_POINT_STYLE, zorder=101)
ax2 = subplot2.ax
ax2.set_title(f"ERA5 —  # Months S-W  Rejection ({accumulation_window}‑Month Window)")

# ------------------ SHARED FIGURE SETTINGS ------------------
# Set figure size (wide)
fig_mpl = ax1.figure
fig_mpl.set_size_inches(18, 6)

# Shared colorbar for the first two panels — use the mappable from panel 1
cbar12 = fig_mpl.colorbar(
    mappable1,
    ax=[ax1, ax2],
    orientation="horizontal",
    location="bottom",    # remove this line if your Matplotlib is older than 3.6
    fraction=0.06,
    pad=0.08,
    ticks=np.arange(1, 13)
)

cbar12.set_label("Number of months (1–12)")

decorate_fig(fig, title = "") # TODO Make colorbar discrete

**TODO point-to-make**: the above is the SW quality flag, regionally (this should be below the time series quality flag comparison). What we have seen is that for a region in Yemen, the indices seem to agree with ERA5-D before zero-precipitation adjustment rather than afterwards?

You can also import the quality flags from ERA5-Drought, for every calendar month, in the reference period, per accumulation period. 

You are unable to send an API request to download this data for all accumulation periods at once. If you try to do so, you will find that your xarray only contains the data for the last accumulation period (in this case, the 48-month window). 

We have written a simple script that sends separate API requests, concatenating the data into a single xarray, to then use.

In [ ]:
# One location -- reproduced -- keep --
pval_spi, sig_spi = xr_shapiro_test(adjusted_spi_ds, reference_window, index_name = "SPI")
sig_spi_agg = sig_spi.sum(dim = "time") # Aggregate across months.

In [ ]:
quality_spi_all = era5_drought_api_multiple(indicator = "spi", var = "quality", product_type = "reanalysis")
quality_spi_all_agg = quality_spi_all.sum(dim="time")

In [ ]:
quality_spi_addis = quality_spi_all.sel(coords_dict["Addis Ababa, Ethiopia"])
quality_spi_tedd = quality_spi_all.sel(coords_dict["Teddington, United Kingdom"])
quality_spi_denv = quality_spi_all.sel(coords_dict["Denver, USA"])

**TODO point-to-make**: the above is the SW quality flag, at a single location (this should be above the regional quality flag comparison). Except for the 1-month window there seems to be perfect agreement. 

Interesting that for 1-month window we have more months that passs SW quality check than ERA5-Drought. Consistent with the time series comparison of ERA5-D & Reproduce -> as saw that ERA5-Drought was consistently larger, and "spikier". Perhaps our fit is of "higher" quality, or perhaps the test was just less good?

In [ ]:
xnor_result = quality_spi_addis == sig_spi.sel(sites=0)

# --- Config ---
COLORS = {
    "era5": "#1f77b4",
    "repro": "#ff7f0e",
    "match": "#2ca02c",
}

# --- X locations and bar width for 3 bars per group ---
x = np.arange(len(ACCUMULATION_PERIODS))
n_bars = 3
width = 0.8 / n_bars  # keeps total group width ~0.8

fig, ax = plt.subplots(figsize=(10, 5))

for i, p in enumerate(ACCUMULATION_PERIODS):
    # Existing values (assumed scalar fractions)
    era5_val = float(quality_spi_all_agg[f"significance_{p}"].sel(coords_dict["Addis Ababa, Ethiopia"]).values)
    repro_val = float(sig_spi_agg[f"significance_{p}"].sel(sites=0).values)

    # Match fraction from xnor_result (aggregate over time and normalise)
    mean_true = xnor_result[f"significance_{p}"].sum(dim="time", skipna=True)
    mean_true = float(mean_true.compute())

    # Plot 3 bars: left (ERA5D), center (Reproduced), right (Mismatch)
    ax.bar(
        x[i] - width, era5_val, width,
        label="ERA5-Drought" if i == 0 else None, color=COLORS["era5"]
    )
    ax.bar(
        x[i], repro_val, width,
        label="Reproduced" if i == 0 else None, color=COLORS["repro"]
    )
    ax.bar(
        x[i] + width, mean_true, width,
        label="Match frequency" if i == 0 else None, color=COLORS["match"]
    )

# --- Cosmetics ---
ax.set_xticks(x)
ax.set_xticklabels([str(p) for p in ACCUMULATION_PERIODS])
ax.set_xlabel("Accumulation windows (months)")
ax.set_ylabel("Fraction of months in calendar year")
ax.set_ylim(0, 12)
ax.set_title("Fraction of months passing S-W test & match frequency (for each accumulation window).")
ax.legend(frameon=False)
ax.grid(axis="y", alpha=0.2)
plt.tight_layout()
plt.show() 

(section-spei)=
### 4. SPEI comparison
The process of calculating SPEI is very similar to that for SPI, with three major differences:
* SPEI is based on the water balance (precipitation and potential evaporation or evapotranspiration), rather than just precipitation.
* SPEI is based on a log-logistic distribution, rather than a gamma distribution.
* SPEI does not need to be adjusted for months with zero precipitation.

As such, this section is structured similarly to [](section-spi) and can be run entirely independently, but some of the more detailed explanations from said section are left out for brevity this time.

#### 4.1 Download monthly precipitation and potential evaporation data from ERA5
First, the monthly-mean total precipitation (variable `228.128`) and evaporation (variable `251.228`) are downloaded from [_Complete ERA5 global atmospheric reanalysis_ (reanalysis-era5-complete)](https://doi.org/10.24381/cds.143582cf).
More information about the format for these requests is available in the [MARS ERA5 catalogue](https://apps.ecmwf.int/data-catalogues/era5/?class=ea).

When reproducing this notebook yourself, if you have previously run through [](section-spi) and have [caching](https://earthkit-data.readthedocs.io/en/latest/guide/caching.html) enabled in earthkit-data, the previously downloaded precipitation data will be re-used.

In [ ]:
request_era5_precipitation_moda = {
    "param": "228.128",  # Variable: Total precipitation
    "stream": "moda",    # Data stream: Monthly mean reanalysis
} | request_era5_template

request_era5_pev_moda = {
    "param": "251.228",  # Variable: Potential evaporation
    "stream": "moda",    # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and load into xarray
data_era5_waterbalance_cds = ekd.from_source("cds", ID_ERA5, request_era5_precipitation_moda, request_era5_pev_moda)  # Download as field list
data_era5_waterbalance_cds = data_era5_waterbalance_cds.to_xarray(compat="equals")  # Convert to xarray dataset

# Pre-process to desired format
# Note the change in variable name
data_era5_waterbalance_preprocessed = preprocess_era5(data_era5_waterbalance_cds)

# Display in notebook
data_era5_waterbalance_preprocessed

#### 4.2 Calculate water balance over accumulation periods
The water balance represents the net gain or loss of water in an area due to precipitation (gain) and evaporation (loss).
[ECMWF's convention](https://codes.ecmwf.int/grib/param-db/182), as used in ERA5, is that downward fluxes are positive and upward fluxes are negative.
This means that
total precipitation is always positive (or zero),
potential evaporation is always negative (or zero),
and the water balance is simply the sum of the two: _WB = TP + PEV_.

Here, we calculate the water balance per point in the downloaded ERA5 data and accumulate it over the same periods as before (1, 3, 6, 12, 24, 36, and 48 months).

The resulting accumulated time series for the example site defined in [](section-general_setup),
Addis Ababa in Ethiopia in our example,
are displayed in Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-wb-accumulated>`.
This example clearly shows how shorter accumulation periods probe short-term effects such as seasonality,
which are smoothed out in longer periods.

In [ ]:
# Calculate water balance
data_era5_waterbalance = calculate_waterbalance(data_era5_waterbalance_preprocessed)

# Accumulate water balance
data_era5_waterbalance = accumulate(data_era5_waterbalance, "wb")

# Display result
data_era5_waterbalance

The plot below illustrates the seasonal water balance for a country like Ethiopia, where periods of high evapotranspiration exceed precipitation, resulting in a negative water balance. Conversely, during wetter months, precipitation surpasses evapotranspiration, producing a surplus of available water.

In [ ]:
# Display accumulated precipitation at example site
plot_accumulated_waterbalance(data_era5_waterbalance, example_site,
                              glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-wb-accumulated")

```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-wb-accumulated
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-wb-accumulated"

Water balance from ERA5 accumulated over different accumulation periods, for the example site of Addis Ababa, Ethiopia.
```

#### 4.3 Fit log-logistic distribution and compute SPEI
While SPI assumes a gamma distribution for total precipitation,
this assumption does not hold for the water balance.
For SPEI,
various distributions are used in the literature [[Stagge+15](https://doi.org/10.1002/joc.4267)].
While [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)] specifies that ERA5–Drought's SPEI is based on the generalised log-logistic distribution,
following [[Vicente-Serrano+10](https://doi.org/10.1175/2009JCLI2909.1)],
private communication with the authors indicates that a generalised logistic distribution is used instead.

As before, the distribution is fitted to data in the reference window (1991–2020 by default),
Here, we use
[scipy.stats.genlogistic](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.genlogistic.html)
to fit the generalised logistic distribution.
This returns three parameters for each calendar month and accumulation period, namely
shape (_c_), location (_μ_), and scale (_β_).

The observed (accumulated) water balance values along the entire time series
are then compared to the fitted parameters in terms of where they fall on the cumulative distribution function (CDF).
From these CDF values,
SPEI values
for each data point (latitude, longitude, time)
are calculated by transforming to a standard normal distribution.
The zero-precipitation correction from [](section-spi) is not relevant to SPEI.

Note that actually evaluating the CDF
– as opposed to [queueing it up in dask](https://docs.xarray.dev/en/stable/user-guide/dask.html), as done here –
can be slow, especially for a large dataset like global ERA5 precipitation.
As such, if you are interested in a subset of the data,
such as a specific site or period in time,
it may be best to subset your data _before_ calculating the CDF rather than afterwards.
An example of this is provided below in the comparison with ERA5–Drought SPEI.

In [ ]:
# Select data in reference window
data_era5_waterbalance_reference = data_era5_waterbalance.sel(**reference_window)

# Fit generalised logistic distribution
spei_parameters = fit_monthly_spei(data_era5_waterbalance_reference)

# Compute CDF time series
cdf = compute_cdf_spei(data_era5_waterbalance, spei_parameters)

# Calculate SPI from adjusted CDF
spei_reproduced = cdf_to_spei(cdf)

# Display result
spei_reproduced

#### 4.4 SPEI comparison: Time series in example site
Having reproduced the SPEI index from ERA5 precipitation and potential evaporation data following the ERA5–Drought methodology,
we can now compare the results to determine the reproducibility of ERA5–Drought.
We first compare the datasets over time at the example site defined in [](section-general_setup).

As before,
because the global calculation set up in the previous subsections can take a long time to execute,
here we sub-select the downloaded ERA5 data and calculate SPEI for the example site only.
For convenience, the full SPEI pipeline is wrapped into a single `calculate_spei_from_era5` function,
as defined in [](section-code_setup).

In [ ]:
# Select only desired site
data_era5_waterbalance_site = data_era5_waterbalance_preprocessed.sel(**example_site)

# Calculate SPI
spei_reproduced_site = calculate_spei_from_era5(data_era5_waterbalance_site, reference_window=reference_window)

# Persist in memory to speed up analysis – this step may take a few minutes
spei_reproduced_site = spei_reproduced_site.persist()

# Display result
spei_reproduced_site

Next, we download the corresponding data from ERA5–Drought.
Because of size limits on the CDS,
the time series must be downloaded in parts:

In [ ]:
# Main request, based on template and example site
request_era5drought_spei = {
    "variable": ["standardised_precipitation_evapotranspiration_index"],
} | request_era5drought_index | request_site

# Split into batches of up to 20 years each
subrequests_era5drought_spei = batch_requests(request_era5drought_spei, n=20)

# Download data and load into xarray
spei_era5drought = ekd.from_source("cds", ID_ERA5_DROUGHT, *subrequests_era5drought_spei)
spei_era5drought = spei_era5drought.to_xarray(compat="equals", join="outer")
spei_era5drought = spei_era5drought.chunk({"time": -1})  # Full time series in one chunk

# Select only desired site (remove margins)
spei_era5drought = spei_era5drought.sel(**example_site)

# Persist in memory to speed up analysis – this step may take a few minutes
spei_era5drought = spei_era5drought.persist()

# Display result
spei_era5drought

Again, we examine the per-point difference in SPEI between ERA5–Drought and the reproduction for each calendar month and each accumulation period.
We calculate the median difference _Δ_ and median absolute difference _|Δ|_:

In [ ]:
# Calculate SPEI median differences
spei_difference_site = comparison_monthly_statistics(spei_era5drought, spei_reproduced_site)

# Display with style
display_monthly_statistics(spei_difference_site)

**Description of comparisons**

Table for quantitative differences

Median and absolute median difference for each accumulation window across months are then calculated  

**TODO point-to-make**: SPI is only discrepant for certain months (but that this discrepancy is still quite small)- since fitting is by month only.

**TODO point-to-make**: the SPEI-index shows considerably less agreement than the SPI index in the previous section, for the same location. We understand that the fitting distribution in the case of the SPEI index is particularly sensitive to the parameterisation used, and that the parameters that are outputted are highly dependent on the initial "guess" of the parameters. This in the ERA5-D paper.

**Description of figures**

**TODO point-to-make**: SPI for location in Addis Ababa shows excellent agreement across entire time series. Showed that our methodology is consistent with that of ERA5 and reproduceable.  
Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-timeseries>`  
Same seen in different sites (not shown here)  

A confusion matrix is generated to compare drought classifications for the single, above location, between the ERA5‑Drought dataset and the reproduced dataset, capturing any discrepancies. The matrix is computed over the full time period (1940–2024), where each count corresponds to a single monthly timestep.  
Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spi-confusion>`  
**TODO point-to-make**: and that in fact, the indices fall under the same drought severity classification for all timestamps at Addis Ababa.  

**TODO point-to-make**: even though there is considerably less agreement in the SPEI index, the agreement in drought severity for SPEI is not particularly affected, except for a few timestamps where the severity is +- 1 category.

In [ ]:
# Plot time series comparison (absolute values and differences)
plot_time_series_comparison_spei(spei_era5drought, spei_reproduced_site, title_suffix=f" at {label_site}",
                                 glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-timeseries")

In [ ]:
# Plot confusion matrix comparison
# e.g. are all "extremely dry" data in ERA5–Drought also "extremely dry" in the reproduction?
plot_confusion_matrices_spei(spei_era5drought, spei_reproduced_site, title_suffix=f" at {label_site}",
                             glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion")

::::{tab-set}
:::{tab-item} Time series
:sync: timeseries
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-timeseries
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-timeseries"

SPEI time series downloaded from ERA5–Drought and reproduced from ERA5 precipitation data (left) and the difference between the two (right), for the example site of Addis Ababa, Ethiopia.
Colours in the left-hand column correspond to SPEI categories (e.g. "extremely dry") in [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)].
```
:::
:::{tab-item} Confusion matrix
:sync: confusion
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion"

Confusion matrices for SPEI categories from ERA5–Drought vs. reproduced from ERA5, for the example site of Addis Ababa, Ethiopia, in 1940–2024.
```
:::
::::

#### 4.5 SPEI comparison: Regional snapshot
Next, we investigate spatial patterns in SPEI and the difference therein across a wider region around the example site.
This region is defined in [](section-general_setup).
In this example, we look at part of the Horn of Africa using a box of 12° in all directions around Addis Ababa, Ethiopia.
To reduce computing requirements, the comparison is performed for one year only,
here 2024, again defined in [](section-general_setup).

As in the time series comparison,
we subselect the desired data from ERA5 and calculate the corresponding SPEI
and
download SPEI from ERA5–Drought for the desired region and time span:

In [ ]:
# Select only desired site
data_era5_waterbalance_region = data_era5_waterbalance_preprocessed.sel(**example_region)

# Window in which to evaluate SPI
evaluation_window = {"time": slice(f"{snapshot_year}-01-01", f"{snapshot_year}-12-01")}  #  Slice (2024-01-01, 2024-12-01)

# Calculate SPI
spei_reproduced_region = calculate_spei_from_era5(data_era5_waterbalance_region, reference_window=reference_window,
                                                  evaluation_window=evaluation_window)

# Persist in memory to speed up analysis – this step may take a few minutes
spei_reproduced_region = spei_reproduced_region.persist()

# Display result
spei_reproduced_region

In [ ]:
# Main request, based on template and example region
request_era5drought_spei = {
    "variable": ["standardised_precipitation_evapotranspiration_index"],
} | request_era5drought_index | request_region

# Select only desired year
request_era5drought_spei = request_era5drought_spei | {"year": [f"{snapshot_year}"],}

# Download data and load into xarray
spei_era5drought = ekd.from_source("cds", ID_ERA5_DROUGHT, request_era5drought_spei)
spei_era5drought = spei_era5drought.to_xarray(compat="equals", join="outer")
spei_era5drought = spei_era5drought.chunk({"time": -1})  # Full time series in one chunk

# Persist in memory to speed up analysis – this step may take a few minutes
spei_era5drought = spei_era5drought.persist()

# Display result
spei_era5drought

First, we examine the per-point difference in SPI between ERA5–Drought and the reproduction.
This comparison is performed separately for each calendar month and each accumulation period
to reflect the fitting process.
We calculate the median difference _Δ_ and median absolute difference _|Δ|_:

In [ ]:
# Calculate SPEI median differences
spei_difference_region = comparison_monthly_statistics(spei_era5drought, spei_reproduced_region)

# Display with style
display_monthly_statistics(spei_difference_region)

Analysis text

(Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-geospatial>`)
(Figure {numref}`{number} <indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion-region>`)

In [ ]:
plot_geospatial_comparison(spei_era5drought, spei_reproduced_region, var="SPEI", time=f"{snapshot_year}-12-01",
                           domain=domain_region,
                           glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-geospatial")

In [ ]:
# Plot confusion matrix comparison
# e.g. are all "extremely dry" data in ERA5–Drought also "extremely dry" in the reproduction?
plot_confusion_matrices_spei(spei_era5drought, spei_reproduced_region, title_suffix=f" in {label_region}",
                             glue_label="indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion-region")

::::{tab-set}
:::{tab-item} Geospatial
:sync: geospatial
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-geospatial
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-geospatial"

SPEI downloaded from ERA5–Drought (left) vs. reproduced from ERA5 precipitation data (middle) and the difference between the two (right), for the region around the example site of Addis Ababa, Ethiopia.
Only one month (December 2024) is displayed.
Colours correspond to SPEI categories (e.g. "extremely dry") in [[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)].
```
:::
:::{tab-item} Confusion matrix
:sync: confusion
```{glue:figure} indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion-region
:name: "indicator_derived-drought-historical-monthly_consistency_q02_fig-spei-confusion-region"

Confusion matrices for SPEI categories from ERA5–Drought vs. reproduced from ERA5, across the region around the example site of Addis Ababa, Ethiopia.
Combines data for all months within one year (2024).
```
:::
::::

#### 4.6 Quality flags: Shapiro–Wilk normality test

In [ ]:
# Setup: Request for each accumulation period
request_p_normality = {
    "variable": ["test_for_normality_spei"],
} | request_region | request_era5drought_flag

requests_per_accumulation_period = {period: {"accumulation_period": [str(period)],}
                                    for period in ACCUMULATION_PERIODS}

subrequests_era5drought_p_normality = {period: (req | request_p_normality)
                                       for period, req in requests_per_accumulation_period.items()}

# Download data in individual requests
p_normality_era5drought = {period: ekd.from_source("cds", ID_ERA5_DROUGHT, subreq)  # Download as field lists
                           for period, subreq in subrequests_era5drought_p_normality.items()}

# Pre-process: Convert to xarray, handle variable names, month dimension
p_normality_era5drought = preprocess_era5drought_qualityflag(p_normality_era5drought, "SPEI")

# Display in notebook
p_normality_era5drought

In [ ]:
# Select latitude and longitude box (40 x 40 here)
dom_lon_min, dom_lon_max = 30, 53  
dom_lat_min, dom_lat_max = -5, 18

lon_min, lon_max = 30, 31  
lat_min, lat_max = 17, 18

# Subset (latitude is descending in ERA5) - should this be wrapped in a function now?
ds_loc = era5_wb_monthly.sel(
    latitude=slice(dom_lat_max, dom_lat_min),
    longitude=slice(dom_lon_min, dom_lon_max))

# Accumulate precipitations
wb_example_global = accumulate(ds_loc, var = "wb") 

# Select reference precipitation window.
wb_example_global_reference = wb_example_global.sel(**reference_window) 

# Fit distribution to reference window, extract parameters.
params_fitted_global = fit_monthly_spei(wb_example_global_reference)

# Calculate CDF  with parameters.
cdf_global = compute_monthly_spei(wb_example_global, params_fitted_global)

spei_global = cdf_to_znorm_transform(cdf_global, "SPEI")

In [ ]:
pval_spei_global, sig_spei_global = xr_shapiro_test(spei_global, reference_window, index_name = "SPEI")
sig_spei_agg_global = sig_spei_global.sum(dim = "time") # Aggregate across months.

quality_spei_all = era5_drought_api_multiple(indicator = "spei", var = "quality", product_type = "reanalysis")
quality_spei_all_agg = quality_spei_all.sum(dim="time")

accumulation_window = 1

# --- Colormap as you already have ---
beige_to_red = LinearSegmentedColormap.from_list(
    "beige_to_red",
    ["#fbeec2", "#d47a57", "#8b0000"]   # beige → soft red → dark red
)

ZERO_PRECIP_STYLE      = Style(cmap=beige_to_red, vmin=1,  vmax=12,  normalize=False)
BLACK_POINT_STYLE      = Style(cmap=ListedColormap(["black"]), vmin=0, vmax=0.99, normalize=False)
ZERO_PRECIP_DIFF_STYLE = Style(cmap="RdBu_r", vmin=-12, vmax=12, normalize=False)

# ------------------ LEFT PANEL ------------------
da1 = sig_spei_agg_global[f"significance_{accumulation_window}"] # Reproduced
non_ones1 = da1.where(da1 != 0)   # all values except 0
ones1     = da1.where(da1 == 0)   # only values equal to 0

# ------------------ MIDDLE PANEL ------------------
da2 = quality_spei_all_agg[f"significance_{accumulation_window}"].sel(
    lat=slice(lat_max, lat_min),
    lon=slice(lon_min, lon_max)) # ERA5-D

non_ones2 = da2.where(da2 != 0)
ones2     = da2.where(da2 == 0)

In [ ]:
horn = ekp.geo.domains.Domain(
    bbox=[dom_lon_min, dom_lon_max, dom_lat_min, dom_lat_max],
    name="Horn of Africa",
)

# Create figure with 3 columns
fig = ekp.Figure(rows=1, columns=2)

# ---------- Subplot 1 ----------
subplot1 = fig.add_map(domain=horn, row=0, column=0)
mappable1 = subplot1.grid_cells(non_ones1.persist(), style=ZERO_PRECIP_STYLE)
subplot1.grid_cells(ones1.persist(), style=BLACK_POINT_STYLE, zorder=101)
ax1 = subplot1.ax
ax1.set_title(f"Reproduced — # Months S-W  Rejection ({accumulation_window}‑Month Window)")

# ---------- Subplot 2 ----------
subplot2 = fig.add_map(domain=horn, row=0, column=1)
mappable2 = subplot2.grid_cells(non_ones2.persist(), style=ZERO_PRECIP_STYLE)
subplot2.grid_cells(ones2.persist(), style=BLACK_POINT_STYLE, zorder=101)
ax2 = subplot2.ax
ax2.set_title(f"ERA5 —  # Months S-W  Rejection ({accumulation_window}‑Month Window)")

# ------------------ SHARED FIGURE SETTINGS ------------------
# Set figure size (wide)
fig_mpl = ax1.figure
fig_mpl.set_size_inches(18, 6)

# Shared colorbar for the first two panels — use the mappable from panel 1
cbar12 = fig_mpl.colorbar(
    mappable1,
    ax=[ax1, ax2],
    orientation="horizontal",
    location="bottom",    # remove this line if your Matplotlib is older than 3.6
    fraction=0.06,
    pad=0.08,
    ticks=np.arange(1, 13)
)

cbar12.set_label("Number of months (1–12)")

decorate_fig(fig, title = "") # TODO Make colorbar discrete

**TODO point-to-make**: have shown that region around Yemen seems to disagreement in SW quality flag values. This is to be expected since the calculated dataset is not in fact in good agreement with ERA5-Drought. Check at another location with more precipitation (somewhere in mainland europe maybe).

Quality flags are imported from ERA5-Drought, for every calendar month, in the reference period, per accumulation period for the SPEI index.

You are unable to send an API request to download this data for all accumulation periods at once. We have written a simple script that sends separate API requests, concatenating the data into a single xarray, to then use.

In [ ]:
era5_quality_spei_global = era5_drought_api_multiple(indicator = "spei", var = "quality", product_type = "reanalysis")
era5_quality_spei_global_agg = era5_quality_spei_global.sum(dim="time")

era5_quality_spei_addis = era5_quality_spei_global.sel(sites_dict["Addis Ababa, Ethiopia"])
era5_quality_spei_tedd = era5_quality_spei_global.sel(sites_dict["Teddington, United Kingdom"])
era5_quality_spei_denv = era5_quality_spei_global.sel(sites_dict["Denver, USA"])

era5_sig_spei_addis_agg = era5_quality_spei_global_agg.sel(sites_dict["Addis Ababa, Ethiopia"])
era5_sig_spei_tedd_agg = era5_quality_spei_global_agg.sel(sites_dict["Teddington, United Kingdom"])
era5_sig_spei_denv_agg = era5_quality_spei_global_agg.sel(sites_dict["Denver, USA"])

The calculated Shapiro-Wilks significance values are then compared with those from ERA5.

In [ ]:
# XNOR Result 
xnor_spei_result = era5_quality_spei_addis == sig_spei

#TODO: Wrap up all code in the function.

# --- Config ---
COLORS = {
    "era5": "#1f77b4",
    "repro": "#ff7f0e",
    "match": "#2ca02c",
}

# --- X locations and bar width for 3 bars per group ---
x = np.arange(len(ACCUMULATION_PERIODS))
n_bars = 3
width = 0.8 / n_bars  # keeps total group width ~0.8

fig, ax = plt.subplots(figsize=(10, 5))

for i, p in enumerate(ACCUMULATION_PERIODS):
    # Existing values (assumed scalar fractions)
    era5_val = float(era5_sig_spei_addis_agg[f"significance_{p}"].values)
    repro_val = float(sig_spei_aggregated[f"significance_{p}"].values)

    # Mismatch fraction from xnor_result (aggregate over time)
    mean_true = xnor_spei_result[f"significance_{p}"].sum(dim="time", skipna=True)
    mean_true = float(mean_true.compute())

    # Plot 3 bars: left (ERA5D), center (Reproduced), right (Mismatch)
    ax.bar(
        x[i] - width, era5_val, width,
        label="ERA5-Drought" if i == 0 else None, color=COLORS["era5"]
    )
    ax.bar(
        x[i], repro_val, width,
        label="Reproduced" if i == 0 else None, color=COLORS["repro"]
    )
    ax.bar(
        x[i] + width, mean_true, width,
        label="Match frequency" if i == 0 else None, color=COLORS["match"]
    )

# --- Cosmetics ---
ax.set_xticks(x)
ax.set_xticklabels([str(p) for p in ACCUMULATION_PERIODS])
ax.set_xlabel("Accumulation windows (months)")
ax.set_ylabel("# of months")
ax.set_title("Significance  and Match Frequency by Accumulation Period")
ax.legend(frameon=False)
ax.set_ylim([-12, 12])
ax.grid(axis="y", alpha=0.2)
plt.tight_layout()
plt.show()

**TODO point-to-make**: SW quality flag does show good agreement however at single location.

#### 4.7 Comparison of ERA5-Drought & Reproduced SPEI-index with quality flags (Addis Ababa, Ethiopia)

In [ ]:
spei_ds_addis_masked = apply_sw_quality_mask(era5_quality_spei_addis, spei_ds.sel(sites=0), "SPEI")
era5_masked = apply_sw_quality_mask(era5_quality_spei_addis, spei_addis_point, "SPEI")

In [ ]:
plot_index_comparison_side_by_side("SPEI", era5_masked, spei_ds_addis_masked, spei_categories) # takes a while

**TODO point-to-make**: make comparison with quality flags- perhaps look at a location where there are more failed months? Otherwise, get rid since only 1 month fails SW test.

(section-ensemble)=
### 5. Ensemble comparison
Once again, calculating the SPI-index for all 10 ensemble members are the same as calculating the SPI index for one member, with the only modification being that there is an extra coordinate, "number" that identifies each ensemble member.


Download monthly ensemble precipitation data, as before.

In [ ]:
request_era5_precipitation_edmo = {
    "param": "228.128",       # Variable: Total precipitation
    "stream": "edmo",         # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and convert to desired format
era5_monthly_mean_ens = ekd.from_source("cds", ID_ERA5, request_era5_precipitation_edmo)  # Download as field list
era5_monthly_mean_ens = era5_monthly_mean_ens.to_xarray(compat="equals")  # Convert to xarray dataset
era5_tp_monthly_mean_ens = rechunk(era5_monthly_mean_ens)  # Re-chunk for speed gain in fitting
era5_tp_monthly_mean_ens  # Display in notebook

And we also download the monthly-mean total potential evaporation data from the ERA5 reanalysis.

In [ ]:
request_era5_pev_edmo = {
    "param": "251.228",       # Variable: Potential evaporation (pev)
    "stream": "edmo",         # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and convert to desired format
era5_pev_monthly_mean_ens = ekd.from_source("cds", ID_ERA5, request_era5_pev_edmo)  # Download as field list
era5_pev_monthly_mean_ens = era5_pev_monthly_mean_ens.to_xarray(compat="equals")  # Convert to xarray dataset
era5_pev_monthly_mean_ens = rechunk(era5_pev_monthly_mean_ens)  # Re-chunk for speed gain in fitting

Precipitation is accumulated for all the accumulation periods in ERA5.

In [ ]:
example_site = {"latitude": 9.25, "longitude": 40.5,}

In [ ]:
# Select the grid point and time slice for Ethiopia
precipitation_example_site = era5_tp_monthly_mean_ens.sel(**example_site)

# Perform the accumulation for each accumulation period
precipitation_ens_example_site = accumulate(precipitation_example_site, "tp")

As is potential evaporaiton, for all the accumulation periods in ERA5.

In [ ]:
# Select the grid point and time slice for Ethiopia
pev_example_site = era5_pev_monthly_mean_ens.sel(**example_site)

# Perform the accumulation for each accumulation period
pev_ens_example_site = accumulate(pev_example_site, "pev")

The "water balance" is calculated, ensuring once again the -ve of the PET, as discussed earlier.  

In [ ]:
wb_ens_example_site = xr.Dataset(
    {
        f"wb_{var.replace('tp_', '')}": precipitation_ens_example_site[var]
                                       + pev_ens_example_site[var.replace("tp_", "pev_")]
        for var in precipitation_ens_example_site.data_vars # loop through every variable.
        if var.startswith("tp_") and var.replace("tp_", "pev_") in pev_ens_example_site.data_vars
    }
)

Gamma distribution is fitted for each calendar month (12) for each ensemble member (here, 10), with their parameters calculated.

In [ ]:
# Fit log logistic distributions
gamma_ens_params_fitted = fit_monthly_spi(precipitation_ens_example_site.sel(**reference_window))

Log-logistic distribution is fitted for each calendar month (12) for each ensemble member (here, 10), with their parameters calculated.

In [ ]:
log_logistic_ens_params_fitted = fit_monthly_spei(wb_ens_example_site.sel(**reference_window))

SPI is calculated for each accumulation period and each ensemble member from the fitted gamma distribution parameters.

In [ ]:
# Compute SPI series -- new function
cdf_ens_ds = compute_monthly_spi(precipitation_ens_example_site, gamma_ens_params_fitted)

SPEI is calculated for each accumulation period and each ensemble member from the fitted log-logistic distribution parameters.

In [ ]:
# Compute SPEI series -- new function
cdf_wb_ens_ds = compute_monthly_spei(wb_ens_example_site, log_logistic_ens_params_fitted)

Quality control is tested with the Shapiro-Wilks test on calculated ensemble SPI data.

In [ ]:
ens_spi_stat, ens_spi_pval, ens_spi_sig = xr_shapiro_test(spi_ens_ds, reference_window)

In [ ]:
quality_spi_ens_all = era5_drought_api_multiple(indicator = "spi", var = "quality", product_type = "ensemble_members")

Quality control using the Shapiro-Wilks test on calculated ensemble SPEI data.

In [ ]:
ens_spei_stat, ens_spei_pval, ens_spei_sig = xr_shapiro_test(spei_ens_ds, reference_window)

In [ ]:
quality_spei_ens_all = era5_drought_api_multiple(indicator = "spei", var = "quality", product_type = "ensemble_members")

Reading in ensemble SPI 

In [ ]:
era5_spi_ens = era5_drought_index_multiple("SPI", accum_period = [1,3])
era5_spi_ens = era5_spi_ens.sel(lat = 9.25, lon = 40.5)

Time-series comparison of SPI at one location 

In [ ]:
for m in range(0, 10):  #### patch this code so titles show which plot is which ensemble member. is obvious but for full thing?
    plot_index_comparison_side_by_side("SPI", era5_spi_ens.sel(number=m), spi_ens_ds.sel(number=m), accumulation_periods = [1]) 

In [ ]:
# TODO: Have a table for each accumulation window maybe.

Reading in ensemble SPEI

In [ ]:
era5_spei_ens = era5_drought_index_multiple("SPEI", accum_period = [3, 24, 48])
era5_spei_ens = era5_spei_ens.sel(lat = 9.25, lon = 40.5) ### data needs to be downloaded overnight, couldn't do it.

Time-series comparison of SPEI at one location 

In [ ]:
for m in range(0, 10):  #### patch this code so titles show which plot is which ensemble member. is obvious but for full thing?
    plot_index_comparison_side_by_side("SPEI", era5_spei_ens.sel(number=m), spei_ens_ds.sel(number=m), [3, 24, 48])

**TODO point-to-make**: each member from the drought-index ensemble is in fact reproduceable from each member from the precipitation / potential evaporation ensemble.

(section-conclusion)=
### 6. Conclusions
**SPI**- As shown in Section 2, the C3S ERA5 Drought SPI‑Indicator dataset and its manually reproduced counterpart are highly consistent. This agreement is evident across the full time series at a point location in Addis Ababa, Ethiopia, as well as regionally over the Horn of Africa. The median difference and median absolute difference are both close to 0, and the vast majority of pixels exhibit a near‑zero difference (defined here as |Δ| ≤ ε with ε = 1×10⁻⁵ to avoid floating‑point artefacts) across all comparisons.

Furthermore, the "probability of zero precipitation" quality flag was also shown to be reproducible, globally, with no discrepancy. Since the Shapiro-Wilks quality flag is dependent on the dataset after statistical fitting, there was discrepancy at certain months, for Addis Ababa, and other locations...

We also examined discrepancies in drought‑severity categorisation between the C3S ERA5‑Drought dataset and the manually reproduced counterpart at individual locations, including Addis Ababa, London, and Denver. While the severity classifications were generally consistent between ERA5‑Drought and the reproduced dataset, there were instances at specific timestamps where the two SPI indicators diverged sufficiently to fall into different categories.

Overall, 

**SPEI**- Following on from Section 3, 

**SPI/SPEI Ensemble**- Lastly, as was shown in Section 4, 

## ℹ️ If you want to know more

### Key resources
The CDS catalogue entries for the data used were:
* Complete ERA5 global atmospheric reanalysis: [reanalysis-era5-complete](https://doi.org/10.24381/cds.143582cf)
* ERA5 hourly data on single levels from 1940 to present: [reanalysis-era5-single-levels](https://doi.org/10.24381/cds.adbb2d47)
* Monthly drought indices from 1940 to present derived from ERA5 reanalysis: [derived-drought-historical-monthly](https://doi.org/10.24381/9bea5e16)

Code libraries used:
* [earthkit](https://github.com/ecmwf/earthkit)
  * [earthkit-data](https://github.com/ecmwf/earthkit-data)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)
    
### References

[[ECIU+25](https://eciu.net/analysis/reports/2025/estimated-financial-losses-faced-by-uk-farmers-due-dry-weather-impacts-on-key-arable-crops)] Energy & Climate Intelligence Unit, ‘Estimated financial losses faced by UK farmers due dry weather impacts on key arable crops’, Energy & Climate Intelligence Unit, London, United Kingdom, Dec. 2025.

[[IPCC+23](https://doi.org/10.59327/IPCC/AR6-9789291691647)] IPCC, ‘Climate Change 2023: Synthesis Report. Contribution of Working Groups I, II and III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change’, Intergovernmental Panel on Climate Change (IPCC), Geneva, Switzerland, Jul. 2023. doi: 10.59327/IPCC/AR6-9789291691647.

[[UNICEF+24](https://www.unicef.org/media/165191/file/LACR-Flash-Update-11-November-2024.pdf)] UNICEF, ‘Latin America and Caribbean Region Flash Update No. 2 (Climate-related crisis in the Amazon Region)’, UNICEF, Nov. 2024.

[[Franch-Pardo+25](https://doi.org/10.48088/ejg.i.fra.16.2.286.297)] I. Franch-Pardo, P. A. F. Puig, and A. Cerdà, ‘Geospatial Technologies in Crisis Response: Analyzing the 2024 Floods in Valencia, Spain’, European Journal of Geography, vol. 16, no. 2, pp. 286–297, Aug. 2025, doi: 10.48088/ejg.i.fra.16.2.286.297.

[[McKee+93](https://climate.colostate.edu/pdfs/relationshipofdroughtfrequency.pdf)] T. B. McKee, N. J. Doesken, and J. Kleist, ‘The relationship of drought frequency and duration to time scales’, in Eighth Conference on Applied Climatology, Anaheim, California, USA, Jan. 1993.

[[Vicente-Serrano+10](https://doi.org/10.1175/2009JCLI2909.1)] S. M. Vicente-Serrano, S. Beguería, and J. I. López-Moreno, ‘A Multiscalar Drought Index Sensitive to Global Warming: The Standardized Precipitation Evapotranspiration Index’, Journal of Climate, vol. 23, no. 7, pp. 1696–1718, Apr. 2010, doi: 10.1175/2009JCLI2909.1.

[[Soci+24](https://doi.org/10.1002/qj.4803)] C. Soci et al., ‘The ERA5 global reanalysis from 1940 to 2022’, Quarterly Journal of the Royal Meteorological Society, vol. 150, no. 764, pp. 4014–4048, Jul. 2024, doi: 10.1002/qj.4803.

[[Hersbach+20](https://doi.org/10.1002/qj.3803)] H. Hersbach et al., ‘The ERA5 global reanalysis’, Quarterly Journal of the Royal Meteorological Society, vol. 146, no. 730, pp. 1999–2049, May 2020, doi: 10.1002/qj.3803.

[[Keune+25](https://doi.org/10.1038/s41597-025-04896-y)] J. Keune, F. Di Giuseppe, C. Barnard, E. Damasio da Costa, and F. Wetterhall, ‘ERA5–Drought: Global drought indices based on ECMWF reanalysis’, Scientific Data, vol. 12, p. 616, Apr. 2025, doi: 10.1038/s41597-025-04896-y.

[[Stagge+15](https://doi.org/10.1002/joc.4267)] J. H. Stagge, L. M. Tallaksen, L. Gudmundsson, A. F. Van Loon, and K. Stahl, ‘Candidate Distributions for Climatological Drought Indices (SPI and SPEI)’, International Journal of Climatology, vol. 35, no. 13, pp. 4027–4040, Feb. 2015, doi: 10.1002/joc.4267.

[[Shapiro+65](https://doi.org/10.1093/biomet/52.3-4.591)] S. S. Shapiro and M. B. Wilk, ‘An analysis of variance test for normality (complete samples)’, Biometrika, vol. 52, no. 3–4, pp. 591–611, Dec. 1965, doi: 10.1093/biomet/52.3-4.591.